In [ ]:
# | default_exp _docusaurus_helper

# Docusaurus Helper

In [ ]:
# | export

import itertools
import re
import ast
import types
from inspect import Signature, getmembers, isclass, isfunction, signature, ismethod, getsource, Parameter
from pathlib import Path
from typing import *
from urllib.parse import urljoin
from functools import lru_cache

import typer
from docstring_parser import parse
from docstring_parser.common import DocstringParam, DocstringRaises, DocstringReturns, Docstring
from nbdev.config import get_config
from nbdev.doclinks import NbdevLookup, patch_name, L, _find_mod
from nbdev_mkdocs.mkdocs import (
    _add_all_submodules,
    _import_all_members,
    _import_functions_and_classes,
    _import_submodules
)

In [ ]:
import sys
import functools
import random
from tempfile import TemporaryDirectory
from contextlib import contextmanager
from abc import abstractmethod
from unittest.mock import patch
import textwrap


import pytest
from pydantic import BaseModel
from aiokafka import ConsumerRecord
from kafka.coordinator.assignors.roundrobin import RoundRobinPartitionAssignor

from fastkafka._components.asyncapi import ConsumeCallable

In [ ]:
# | export


def _get_return_annotation(s: Signature) -> str:
    """Get the return annotation from the function signature.

    Args:
        s: The signature of the function from which the annotations must be extracted.

    Returns:
        The return annotation, or an empty string if not available.

    """
    if s.return_annotation == None or "inspect._empty" in str(s.return_annotation):
        return ""
    if isinstance(s.return_annotation, str):
        return s.return_annotation
    ret_val: str = (
        str(s.return_annotation).replace("typing.", "").replace("NoneType", "None")
        if "typing." in str(s.return_annotation)
        else str(s.return_annotation.__name__)
    )
    return ret_val

In [ ]:
def fixture() -> Callable[[ConsumeCallable], ConsumeCallable]:
    pass

# _signature = signature(fixture)
# actual = _get_return_annotation(_signature)
# expected = "Callable[[ConsumeCallable], ConsumeCallable]"
# print(actual)

# assert actual == expected

In [ ]:

def fixture():
    pass

_signature = signature(fixture)
actual = _get_return_annotation(_signature)
expected = ""
print(actual)

assert actual == expected

In [ ]:

def fixture() -> Callable[[ConsumerRecord], Awaitable[None]]:
    pass

_signature = signature(fixture)
actual = _get_return_annotation(_signature)
expected = "Callable[[aiokafka.structs.ConsumerRecord], Awaitable[None]]"
print(actual)

assert actual == expected

Callable[[aiokafka.structs.ConsumerRecord], Awaitable[None]]


In [ ]:
def fixture() -> Callable[["FastAPI"], AsyncIterator[None]]:
    pass

_signature = signature(fixture)
actual = _get_return_annotation(_signature)
expected = "Callable[[ForwardRef('FastAPI')], AsyncIterator[None]]"
print(actual)

assert actual == expected

Callable[[ForwardRef('FastAPI')], AsyncIterator[None]]


In [ ]:
def fixture() -> "EventMetadata":
    pass

_signature = signature(fixture)
actual = _get_return_annotation(_signature)
expected = "EventMetadata"
print(actual)

assert actual == expected

EventMetadata


In [ ]:
def fixture() -> Optional[str]:
    pass


_signature = signature(fixture)
actual = _get_return_annotation(_signature)
expected = (
    "Union[str, None]"
    if f"{sys.version_info.major}.{sys.version_info.minor}" == "3.8"
    else "Optional[str]"
)
print(actual)

assert actual == expected, expected

Optional[str]


In [ ]:
def fixture() -> Iterable[str]:
    pass

_signature = signature(fixture)
actual = _get_return_annotation(_signature)
expected = "Iterable[str]"
print(actual)

assert actual == expected

Iterable[str]


In [ ]:
def fixture() -> None:
    pass

_signature = signature(fixture)
actual = _get_return_annotation(_signature)
expected = ""
print(actual)

assert actual == expected

In [ ]:
def fixture() -> str:
    pass

_signature = signature(fixture)
actual = _get_return_annotation(_signature)
expected = "str"
print(actual)

assert actual == expected

str


In [ ]:
def fixture() -> List[int]:
    pass

_signature = signature(fixture)
actual = _get_return_annotation(_signature)
expected = "List[int]"
print(actual)

assert actual == expected

List[int]


In [ ]:
def fixture() -> Dict[str, Any]:
    pass

_signature = signature(fixture)
actual = _get_return_annotation(_signature)
expected = "Dict[str, Any]"
print(actual)

assert actual == expected

Dict[str, Any]


In [ ]:

def fixture() -> Union[str, List[str]]:
    pass

_signature = signature(fixture)
actual = _get_return_annotation(_signature)
expected = "Union[str, List[str]]"
print(actual)

assert actual == expected

Union[str, List[str]]


In [ ]:
class A:
    pass


def fixture() -> A:
    pass

_signature = signature(fixture)
actual = _get_return_annotation(_signature)
expected = "A"
print(actual)

assert actual == expected

A


In [ ]:


def fixture() -> Callable[[], Any]:
    pass

_signature = signature(fixture)
actual = _get_return_annotation(_signature)
expected = "Callable[[], Any]"
print(actual)

assert actual == expected

Callable[[], Any]


In [ ]:
# | export

def _get_param_annotation(param: Parameter) -> str:
    """Get the annotation of a function parameter.

    Args:
        param: The function parameter object.

    Returns:
        The parameter annotation, or an empty string if not available.

    """
    if "typing." in str(param.annotation):
        return f'`{str(param.annotation).replace("typing.", "")}`'
    else:
        return (
            "" if param.annotation.__name__ == "_empty" else f'`{param.annotation.__name__}`'
        )

In [ ]:
def f(a: int, b, c: Dict[str, int]):
    pass

s = signature(f)
actual = []
expected = ['`int`', '', '`Dict[str, int]`']
for param in s.parameters.values():
    actual.append(_get_param_annotation(param))
    
print(actual)
assert sorted(actual) == sorted(expected)

['`int`', '', '`Dict[str, int]`']


In [ ]:
# | export


def _get_params_annotation(s: Signature) -> Dict[str, Dict[str, str]]:
    """Get the annotations along with its default values for the parameters of the symbol.

    Args:
        s: The signature of the function from which the annotations must be extracted.

    Returns:
        The parameter annotations along with its default value.
    """
    return {
        f"{param.name}": {
            "type": _get_param_annotation(param),
            "default": "*required*"
            if param.default is param.empty
            else f"`{param.default}`",
        }
        for param in s.parameters.values()
    }

In [ ]:
def fixture(
    arg_1: int,
    *,
    arg_2: str = "default_string",
    arg_3: Dict[str, int] = {},
    arg_4: Optional[float] = None,
    arg_5: Tuple[int, str, float] = (1, "string", 2.0),
    arg_6: List[Union[int, str]] = [1, "string"],
    arg_7: Set[int] = {1, 2, 3},
    arg_8: str = "string",
    arg_9: Dict[str, str] = {},
    arg_10 = None
) -> str:
    pass


_signature = signature(fixture)
actual = _get_params_annotation(_signature)

_optional_type = "Union[float, NoneType]" if f"{sys.version_info.major}.{sys.version_info.minor}" == "3.8" else "Optional[float]"
expected = {
    "arg_1": {"type": "`int`", "default": "*required*"},
    "arg_2": {"type": "`str`", "default": "`default_string`"},
    "arg_3": {"type": "`Dict[str, int]`", "default": "`{}`"},
    "arg_4": {"type": f"`{_optional_type}`", "default": "`None`"},
    "arg_5": {"type": "`Tuple[int, str, float]`", "default": "`(1, 'string', 2.0)`"},
    "arg_6": {"type": "`List[Union[int, str]]`", "default": "`[1, 'string']`"},
    "arg_7": {"type": "`Set[int]`", "default": "`{1, 2, 3}`"},
    "arg_8": {"type": "`str`", "default": "`string`"},
    "arg_9": {"type": "`Dict[str, str]`", "default": "`{}`"},
    "arg_10": {"type": "", "default": "`None`"},
}
display(actual)
assert actual == expected

{'arg_1': {'type': '`int`', 'default': '*required*'},
 'arg_2': {'type': '`str`', 'default': '`default_string`'},
 'arg_3': {'type': '`Dict[str, int]`', 'default': '`{}`'},
 'arg_4': {'type': '`Optional[float]`', 'default': '`None`'},
 'arg_5': {'type': '`Tuple[int, str, float]`',
  'default': "`(1, 'string', 2.0)`"},
 'arg_6': {'type': '`List[Union[int, str]]`', 'default': "`[1, 'string']`"},
 'arg_7': {'type': '`Set[int]`', 'default': '`{1, 2, 3}`'},
 'arg_8': {'type': '`str`', 'default': '`string`'},
 'arg_9': {'type': '`Dict[str, str]`', 'default': '`{}`'},
 'arg_10': {'type': '', 'default': '`None`'}}

In [ ]:
# | export


def _generate_parameters_table(
    symbol_annotations: Dict[str, Union[Dict[str, str], str]],
    section_items: Union[List[DocstringParam]],
    section_name: str,
) -> str:
    """Generate parameter table in markdown format
    
    Args:
        symbol_annotations: Symbol annotations along with its default value
        section_items: The parameter section of a parsed docstring
        section_name: The name of the section

    Returns:
        The parameters of a symbol in markdown-formatted string
    """
    nl = "\n"
    _section_template = (
        "|  Name | Type | Description | Default |\n|---|---|---|---|\n{section_body}\n"
    )
    section_body = "".join(
        [
            f'| `{section.arg_name}` | {symbol_annotations["parameters"][section.arg_name]["type"]} | {section.description.replace(nl, "")} | {symbol_annotations["parameters"][section.arg_name]["default"]} |\n' # type: ignore
            if section.arg_name in symbol_annotations["parameters"]
            else ""
            for section in section_items
        ]
    )
    return f"**{section_name}**:\n\n" + _section_template.format(
        section_body=section_body,
    )

In [ ]:
params_dict = {
    "name": {"type": "str", "default": "*required*"},
    "age": {"type": "int", "default": "*required*"},
}
return_dict = "str"
signature_dict = {"parameters": params_dict, "return": return_dict}

fixture_docstring = parse("""This is a docstring for a sample function.

    Args:
        name: name of the person
        age: age of the person
    """)

actual = _generate_parameters_table(
    signature_dict, fixture_docstring.params, "Parameters"
)

expected = """**Parameters**:

|  Name | Type | Description | Default |
|---|---|---|---|
| `name` | str | name of the person | *required* |
| `age` | int | age of the person | *required* |

"""
print(actual)
assert actual == expected

**Parameters**:

|  Name | Type | Description | Default |
|---|---|---|---|
| `name` | str | name of the person | *required* |
| `age` | int | age of the person | *required* |




In [ ]:
# | export


def _generate_return_and_raises_table(
    symbol_annotations: Dict[str, Union[Dict[str, str], str]],
    section_items: Union[List[DocstringReturns], List[DocstringRaises]],
    section_name: str,
) -> str:
    """Generate return and raises table in markdown format
    
    Args:
        symbol_annotations: Symbol annotations along with its default value
        section_items: The parameter section of a parsed docstring
        section_name: The name of the section
        
    Returns:
        The return and raises section of a symbol in markdown-formatted string
    """
    nl = "\n"
    _section_template = "|  Type | Description |\n|---|---|\n{section_body}\n"
    section_body = "".join(
        [
            f'| `{symbol_annotations["return"] if section_name == "Returns" else section.type_name}` | {section.description.replace(nl, "")} |\n' # type: ignore
            for section in section_items
        ]
    )
    return f"**{section_name}**:\n\n" + _section_template.format(
        section_body=section_body,
    )

In [ ]:
params_dict = {
    "name": {"type": "str", "default": "*required*"},
    "age": {"type": "int", "default": "*required*"},
}
return_dict = "str"
signature_dict = {"parameters": params_dict, "return": return_dict}

fixture_docstring = parse("""This is a docstring for a sample function.

    Returns:
        A formatted string
    """)

actual = _generate_return_and_raises_table(
    signature_dict, fixture_docstring.many_returns, "Returns"
)

expected = """**Returns**:

|  Type | Description |
|---|---|
| `str` | A formatted string |

"""
print(actual)
assert actual == expected, actual

**Returns**:

|  Type | Description |
|---|---|
| `str` | A formatted string |




In [ ]:
params_dict = {
    "name": {"type": "str", "default": "*required*"},
    "age": {"type": "int", "default": "*required*"},
}
return_dict = "str"
signature_dict = {"parameters": params_dict, "return": return_dict}

fixture_docstring = parse("""This is a docstring for a sample function.

    Raises:
        ValueError: If name is not a string
        TypeError: If type is invalid
        KeyError: If key name is invalid
    """)

actual = _generate_return_and_raises_table(
    signature_dict, fixture_docstring.raises, "Exceptions"
)

expected = """**Exceptions**:

|  Type | Description |
|---|---|
| `ValueError` | If name is not a string |
| `TypeError` | If type is invalid |
| `KeyError` | If key name is invalid |

"""
print(actual)
assert actual == expected, actual

**Exceptions**:

|  Type | Description |
|---|---|
| `ValueError` | If name is not a string |
| `TypeError` | If type is invalid |
| `KeyError` | If key name is invalid |




In [ ]:
# | export


def _format_docstring_section_items(
    symbol_annotations: Dict[str, Union[Dict[str, str], str]],
    section_items: Union[
        List[DocstringParam], List[DocstringReturns], List[DocstringRaises]
    ],
    section_name: str,
) -> str:
    """Format the docstring sections in a table format
    
    Args:
        symbol_annotations: Symbol annotations along with its default value
        section_items: The parameter section of a parsed docstring
        section_name: The name of the section
        
    Returns:
        The docstring sections of the symbol in markdown-formatted string
    """
    if section_name == "Parameters":
        return _generate_parameters_table(symbol_annotations, section_items, section_name) # type: ignore
    else:
        return _generate_return_and_raises_table(symbol_annotations, section_items, section_name) # type: ignore

In [ ]:
params_dict = {
    "name": {"type": "str", "default": "*required*"},
    "age": {"type": "int", "default": "*required*"},
}
return_dict = "str"
signature_dict = {"parameters": params_dict, "return": return_dict}

fixture_docstring = parse("""This is a docstring for a sample function.

    It can contain multiple lines and can include *markdown* syntax.

    Args:
        name: name of the person
        age: age of the person
        some_param: some_param of the person

    Returns:
        A formatted string

    Raises:
        ValueError: If name is not a string
    """)

actual = _format_docstring_section_items(
    signature_dict, fixture_docstring.params, "Parameters"
)

expected = """**Parameters**:

|  Name | Type | Description | Default |
|---|---|---|---|
| `name` | str | name of the person | *required* |
| `age` | int | age of the person | *required* |

"""
print(actual)
assert actual == expected

**Parameters**:

|  Name | Type | Description | Default |
|---|---|---|---|
| `name` | str | name of the person | *required* |
| `age` | int | age of the person | *required* |




In [ ]:
params_dict = {
    "name": {"type": "str", "default": "*required*"},
    "age": {"type": "int", "default": "*required*"},
    "some_param": {"type": "Dict[str, int]", "default": "`{}`"},
}
return_dict = "str"
signature_dict = {"parameters": params_dict, "return": return_dict}

fixture_docstring = parse("""This is a docstring for a sample function.

    It can contain multiple lines and can include *markdown* syntax.

    Args:
        name: name of the person
        age: age of the person
        some_param: some_param of the person

    Returns:
        A formatted string

    Raises:
        ValueError: If name is not a string
    """)

actual = _format_docstring_section_items(
    signature_dict, fixture_docstring.params, "Parameters"
)

expected = """**Parameters**:

|  Name | Type | Description | Default |
|---|---|---|---|
| `name` | str | name of the person | *required* |
| `age` | int | age of the person | *required* |
| `some_param` | Dict[str, int] | some_param of the person | `{}` |

"""
print(actual)
assert actual == expected

**Parameters**:

|  Name | Type | Description | Default |
|---|---|---|---|
| `name` | str | name of the person | *required* |
| `age` | int | age of the person | *required* |
| `some_param` | Dict[str, int] | some_param of the person | `{}` |




In [ ]:
actual = _format_docstring_section_items(
    signature_dict, fixture_docstring.many_returns, "Returns"
)

expected = """**Returns**:

|  Type | Description |
|---|---|
| `str` | A formatted string |

"""
print(actual)
assert actual == expected

**Returns**:

|  Type | Description |
|---|---|
| `str` | A formatted string |




In [ ]:
actual = _format_docstring_section_items(
    signature_dict, fixture_docstring.raises, "Exceptions"
)
expected = """**Exceptions**:

|  Type | Description |
|---|---|
| `ValueError` | If name is not a string |

"""
print(actual)
assert actual == expected

**Exceptions**:

|  Type | Description |
|---|---|
| `ValueError` | If name is not a string |




In [ ]:
# | export


def _get_annotation(symbol: Type) -> Dict[str, Union[Dict[str, Dict[str, str]], str]]:
    """Get annotations along with its default value for a symbol
    
    Args:
        symbol: The symbol for which the annotations needs to be extracted
        
    Returns:
        The annotations dict along with its default value
    """
    symbol = symbol.fget if isinstance(symbol, property) else symbol
    symbol_signature = signature(symbol)
    params_dict = _get_params_annotation(symbol_signature)
    return_annotation = _get_return_annotation(symbol_signature)
    return {
        "parameters": params_dict,
        "return": return_annotation
    }

In [ ]:
@property
def fixture(name: str, info: Tuple[int, str, float] = 80, contact: Optional[str] = None) -> str:
    """This is a docstring for a sample function.

    Args:
        name: name of the person
        info: info of the person
        contact: optional contact for the documentation. If None, the
            contact will be set to placeholder values:
            name='Author' url=HttpUrl('https://www.google.com', ) email='noreply@gmail.com'

    """
    pass

_optional_type = "Union[str, NoneType]" if f"{sys.version_info.major}.{sys.version_info.minor}" == "3.8" else "Optional[str]"
expected = {
    "parameters": {
        "name": {"type": "`str`", "default": "*required*"},
        "info": {"type": "`Tuple[int, str, float]`", "default": "`80`"},
        "contact": {"type": f"`{_optional_type}`", "default": "`None`"},
    },
    "return": "str"
}
actual = _get_annotation(fixture)
display(actual)
assert actual == expected

{'parameters': {'name': {'type': '`str`', 'default': '*required*'},
  'info': {'type': '`Tuple[int, str, float]`', 'default': '`80`'},
  'contact': {'type': '`Optional[str]`', 'default': '`None`'}},
 'return': 'str'}

In [ ]:
def fixture(name: str, info: Union[str, List[str]], contact: Optional[str] = "default_email.com"):
    """This is a docstring for a sample function.

    Args:
        name: name of the person
        info: info of the person
        contact: optional contact for the documentation. If None, the
            contact will be set to placeholder values:
            name='Author' url=HttpUrl('https://www.google.com', ) email='noreply@gmail.com'

    """
    pass

_optional_type = "Union[str, NoneType]" if f"{sys.version_info.major}.{sys.version_info.minor}" == "3.8" else "Optional[str]"
expected = {
    "parameters": {
        "name": {"type": "`str`", "default": "*required*"},
        "info": {"type": "`Union[str, List[str]]`", "default": "*required*"},
        "contact": {"type": f"`{_optional_type}`", "default": "`default_email.com`"}
    },
    "return": ""
}
actual = _get_annotation(fixture)
display(actual)
assert actual == expected

{'parameters': {'name': {'type': '`str`', 'default': '*required*'},
  'info': {'type': '`Union[str, List[str]]`', 'default': '*required*'},
  'contact': {'type': '`Optional[str]`', 'default': '`default_email.com`'}},
 'return': ''}

In [ ]:
# | export


def _format_docstring_sections(
    symbol: Type, parsed_docstring: Docstring
) -> str:
    """Format the parsed docstring sections into markdown-formatted table

    Args:
        symbol: The symbol for which to parse the docstring.
        parsed_docstring: A Docstring object

    Returns:
        The markdown-formatted docstring.
    """
    symbol_annotations = _get_annotation(symbol)
    formatted_docstring = ""
    sections = [
        ("Parameters", parsed_docstring.params),
        ("Returns", parsed_docstring.many_returns),
        ("Exceptions", parsed_docstring.raises),
    ]

    for section_name, section_items in sections:
        if len(section_items) > 0:  # type: ignore
            formatted_docstring += _format_docstring_section_items(
                symbol_annotations, section_items, section_name  # type: ignore
            )

    return formatted_docstring

In [ ]:
def fixture(name: str, age: int):
    """
    This is a docstring for a sample function.

    It can contain multiple lines and can include *markdown* syntax.

    Args:
        name: name of the person
        age: age of the person

    Raises:
        ValueError: If name is not a string
        TypeError: If name is not a string
    """
    pass

parsed_docstring = parse(fixture.__doc__)

expected = """**Parameters**:

|  Name | Type | Description | Default |
|---|---|---|---|
| `name` | `str` | name of the person | *required* |
| `age` | `int` | age of the person | *required* |

**Exceptions**:

|  Type | Description |
|---|---|
| `ValueError` | If name is not a string |
| `TypeError` | If name is not a string |

"""

actual = _format_docstring_sections(fixture, parsed_docstring)
print(actual)

assert actual == expected

**Parameters**:

|  Name | Type | Description | Default |
|---|---|---|---|
| `name` | `str` | name of the person | *required* |
| `age` | `int` | age of the person | *required* |

**Exceptions**:

|  Type | Description |
|---|---|
| `ValueError` | If name is not a string |
| `TypeError` | If name is not a string |




In [ ]:
# | export


def _format_free_links(s: str) -> str:
    """Format free links in a given string by adding proper spacing around them.

    Args:
        s: The input string containing free links.

    Returns:
        The modified string with properly formatted free links.
    """
    pattern = r"([\"'])(https?:\/\/[^\s]+)([\"'])"
    ret_val = re.sub(
        pattern, lambda match: f"{match.group(1)} {match.group(2)} {match.group(3)}", s
    )
    return ret_val

In [ ]:
fixture = """
Click [here](https://www.example.com) to explore Example
Learn more at [Example](http://www.example.co.in)
Discover at [https://www.example.edu](https://www.example.edu)
[Example](https://www.example.co.uk) is worth exploring

url=HttpUrl('https://www.google.com', )
url=HttpUrl('http://www.example.net', )
"https://www.google.edu"
"""
expected = """
Click [here](https://www.example.com) to explore Example
Learn more at [Example](http://www.example.co.in)
Discover at [https://www.example.edu](https://www.example.edu)
[Example](https://www.example.co.uk) is worth exploring

url=HttpUrl(' https://www.google.com ', )
url=HttpUrl(' http://www.example.net ', )
" https://www.google.edu "
"""
actual = _format_free_links(fixture)
print(actual)
assert actual == expected


Click [here](https://www.example.com) to explore Example
Learn more at [Example](http://www.example.co.in)
Discover at [https://www.example.edu](https://www.example.edu)
[Example](https://www.example.co.uk) is worth exploring

url=HttpUrl(' https://www.google.com ', )
url=HttpUrl(' http://www.example.net ', )
" https://www.google.edu "



In [ ]:
# | export


def _docstring_to_markdown(symbol: Type) -> str:
    """Converts a docstring to a markdown-formatted string.

    Args:
        symbol: The symbol for which the documentation needs to be generated in markdown format.

    Returns:
        The markdown-formatted docstring.
    """
    parsed_docstring = parse(symbol.__doc__) # type: ignore
    formatted_docstring = f"{parsed_docstring.short_description}\n\n"
    formatted_docstring += (
        f"{parsed_docstring.long_description}\n\n"
        if parsed_docstring.long_description
        else ""
    )
    formatted_docstring += _format_docstring_sections(symbol, parsed_docstring)
    ret_val = _format_free_links(formatted_docstring)

    return ret_val

In [ ]:
def fixture(name: str, age: int, contact: str):
    """This is a docstring for a sample function.

    Args:
        name: name of the person
        age: age of the person
        contact: optional contact for the documentation. If None, the
            contact will be set to placeholder values:
            name='Author' url=HttpUrl('https://www.google.com', ) email='noreply@gmail.com'

    """
    pass

expected = """This is a docstring for a sample function.

**Parameters**:

|  Name | Type | Description | Default |
|---|---|---|---|
| `name` | `str` | name of the person | *required* |
| `age` | `int` | age of the person | *required* |
| `contact` | `str` | optional contact for the documentation. If None, thecontact will be set to placeholder values:name='Author' url=HttpUrl(' https://www.google.com ', ) email='noreply@gmail.com' | *required* |

"""

actual = _docstring_to_markdown(fixture)
print(actual)

assert actual == expected

This is a docstring for a sample function.

**Parameters**:

|  Name | Type | Description | Default |
|---|---|---|---|
| `name` | `str` | name of the person | *required* |
| `age` | `int` | age of the person | *required* |
| `contact` | `str` | optional contact for the documentation. If None, thecontact will be set to placeholder values:name='Author' url=HttpUrl(' https://www.google.com ', ) email='noreply@gmail.com' | *required* |




In [ ]:
def fixture():
    """This is a docstring for a sample function."""
    pass

expected = """This is a docstring for a sample function.

"""

actual = _docstring_to_markdown(fixture)
print(actual)

assert actual == expected

This is a docstring for a sample function.




In [ ]:
def fixture(name: str, age: int) -> str:
    """This is a docstring for a sample function.

    It can contain multiple lines and can include *markdown* syntax.

    Args:
        name: name of the person
        age: age of the person

    Returns:
        A formatted string

    Raises:
        ValueError: If name is not a string
        TypeError: If name is not a string
    """
    pass

expected = """This is a docstring for a sample function.

It can contain multiple lines and can include *markdown* syntax.

**Parameters**:

|  Name | Type | Description | Default |
|---|---|---|---|
| `name` | `str` | name of the person | *required* |
| `age` | `int` | age of the person | *required* |

**Returns**:

|  Type | Description |
|---|---|
| `str` | A formatted string |

**Exceptions**:

|  Type | Description |
|---|---|
| `ValueError` | If name is not a string |
| `TypeError` | If name is not a string |

"""

actual = _docstring_to_markdown(fixture)
print(actual)

assert actual == expected

This is a docstring for a sample function.

It can contain multiple lines and can include *markdown* syntax.

**Parameters**:

|  Name | Type | Description | Default |
|---|---|---|---|
| `name` | `str` | name of the person | *required* |
| `age` | `int` | age of the person | *required* |

**Returns**:

|  Type | Description |
|---|---|
| `str` | A formatted string |

**Exceptions**:

|  Type | Description |
|---|---|
| `ValueError` | If name is not a string |
| `TypeError` | If name is not a string |




In [ ]:
# | export


def _get_submodules(module_name: str) -> List[str]:
    """Get a list of all submodules contained within the module.

    Args:
        module_name: The name of the module to retrieve submodules from

    Returns:
        A list of submodule names within the module
    """
    members = _import_all_members(module_name)
    members_with_submodules = _add_all_submodules(members)
    members_with_submodules_str: List[str] = [
        x[:-1] if x.endswith(".") else x for x in members_with_submodules
    ]
    return members_with_submodules_str

In [ ]:
module_name = "fastkafka"
members_with_submodules = _get_submodules(module_name)
members_with_submodules

['fastkafka',
 'fastkafka.EventMetadata',
 'fastkafka.FastKafka',
 'fastkafka.KafkaEvent',
 'fastkafka.encoder',
 'fastkafka.encoder.AvroBase',
 'fastkafka.encoder.avro_decoder',
 'fastkafka.encoder.avro_encoder',
 'fastkafka.encoder.avsc_to_pydantic',
 'fastkafka.encoder.json_decoder',
 'fastkafka.encoder.json_encoder',
 'fastkafka.executors',
 'fastkafka.executors.DynamicTaskExecutor',
 'fastkafka.executors.SequentialExecutor',
 'fastkafka.testing',
 'fastkafka.testing.ApacheKafkaBroker',
 'fastkafka.testing.LocalRedpandaBroker',
 'fastkafka.testing.Tester']

In [ ]:
# | export


def _load_submodules(
    module_name: str, members_with_submodules: List[str]
) -> List[Type]:
    """Load the given submodules from the module.

    Args:
        module_name: The name of the module whose submodules to load
        members_with_submodules: A list of submodule names to load

    Returns:
        A list of imported submodule objects.
    """
    submodules = _import_submodules(module_name)
    members: List[Tuple[str, Type]] = list(
        itertools.chain(*[_import_functions_and_classes(m) for m in submodules])
    )
    names = [
        y
        for x, y in members
        if f"{y.__module__}.{y.__name__}" in members_with_submodules
    ]
    return names

In [ ]:
module_name = "fastkafka"
members_with_submodules = _get_submodules(module_name)
symbols = _load_submodules(module_name, members_with_submodules)
symbols

[fastkafka.EventMetadata,
 fastkafka.FastKafka,
 fastkafka.KafkaEvent,
 fastkafka.encoder.AvroBase,
 <function fastkafka.encoder.avro_decoder(raw_msg: bytes, cls: pydantic.main.ModelMetaclass) -> Any>,
 <function fastkafka.encoder.avro_encoder(msg: pydantic.main.BaseModel) -> bytes>,
 <function fastkafka.encoder.avsc_to_pydantic(schema: Dict[str, Any]) -> pydantic.main.ModelMetaclass>,
 <function fastkafka.encoder.json_decoder(raw_msg: bytes, cls: pydantic.main.ModelMetaclass) -> Any>,
 <function fastkafka.encoder.json_encoder(msg: pydantic.main.BaseModel) -> bytes>,
 fastkafka.executors.DynamicTaskExecutor,
 fastkafka.executors.SequentialExecutor,
 fastkafka.testing.ApacheKafkaBroker,
 fastkafka.testing.LocalRedpandaBroker,
 fastkafka.testing.Tester]

In [ ]:
# | export


def _get_parameters(_signature: Signature) -> List[str]:
    """Convert a function's signature into a string representation of its parameter list.

    Args:
        _signature: The signature object representing the function's signature.

    Returns:
        A list of strings representing the function's parameters, including their default values if applicable.
    """
    params = [param for param in _signature.parameters.values()]
    ret_val = [
            f"{param.name}"
            if (param.default is param.empty)
            else f"{param.name}='{param.default}'"
            if isinstance(param.default, str)
            else f"{param.name}={param.default}"
            for param in params
        ]
    return ret_val

In [ ]:
def fixture_function(
    arg_1: str, arg_2, arg_3: Union[Dict[str, str], str], arg_4: Optional[int] = 80
) -> str:
    pass


_signature = signature(fixture_function)

expected = ['arg_1', 'arg_2', 'arg_3', 'arg_4=80']
actual = _get_parameters(_signature)

print(actual)
assert actual == expected

['arg_1', 'arg_2', 'arg_3', 'arg_4=80']


In [ ]:
def fixture_function(arg_1: str, arg_2) -> None:
    pass


_signature = signature(fixture_function)

expected = ["arg_1", "arg_2"]
actual = _get_parameters(_signature)

print(actual)
assert actual == expected

['arg_1', 'arg_2']


In [ ]:
def fixture_function(
    arg_1: int,
    *,
    arg_2: str = "default_string",
    arg_3: Dict[str, int] = {},
    arg_4: Optional[float] = None,
    arg_5: Tuple[int, str, float] = (1, "string", 2.0),
    arg_6: List[Union[int, str]] = [1, "string"],
    arg_7: Set[int] = {1,2,3},
    arg_8: str = "string"
) -> None:
    pass

_signature = signature(fixture_function)
params = [param for param in _signature.parameters.values()]
expected = ['arg_1', "arg_2='default_string'", 'arg_3={}', 'arg_4=None', "arg_5=(1, 'string', 2.0)", "arg_6=[1, 'string']", 'arg_7={1, 2, 3}', "arg_8='string'"]
actual = _get_parameters(_signature)

print(actual)
assert actual == expected

['arg_1', "arg_2='default_string'", 'arg_3={}', 'arg_4=None', "arg_5=(1, 'string', 2.0)", "arg_6=[1, 'string']", 'arg_7={1, 2, 3}', "arg_8='string'"]


In [ ]:
# | export


def _format_symbol_definition(
    symbol: Type, params_list: List[str]
) -> str:
    """Format the given symbol parameters by adding a new line and indentation.

    Args:
        symbol: The symbol for which the symbol definition needs to be formatted.
        params_list: A string representation of the parameter list.

    Returns:
        A formatted string representation of the parameters with new lines and indentation.
    """
    parameters = ", ".join(params_list)
    if parameters == "":
        return f"{symbol.__name__}()\n"
    elif len(f"{symbol.__name__}({parameters})") <= 79:
        return f"{symbol.__name__}(\n    {parameters}\n)\n"
    else:
        formatted_parameters = "".join([f"\n    {param}," for param in params_list])
        return f"{symbol.__name__}({formatted_parameters}\n)\n"

In [ ]:
def fixture_function():
    pass


expected = """fixture_function()
"""
_signature = signature(fixture_function)
parameters = _get_parameters(_signature)
parameters

actual = _format_symbol_definition(fixture_function, parameters)
print(actual)
assert actual == expected

fixture_function()



In [ ]:
def fixture_function(arg_1: str) -> str:
    pass


expected = """fixture_function(
    arg_1
)
"""
_signature = signature(fixture_function)
parameters = _get_parameters(_signature)
actual = _format_symbol_definition(fixture_function, parameters)
print(actual)
assert actual == expected

fixture_function(
    arg_1
)



In [ ]:
def fixture_function(
    arg_1: str, *, arg_2, arg_3: Optional[int] = 80
) -> str:
    pass

expected = """fixture_function(
    arg_1, arg_2, arg_3=80
)
"""
_signature = signature(fixture_function)
parameters = _get_parameters(_signature)
actual = _format_symbol_definition(fixture_function, parameters)
print(actual)
assert actual == expected

fixture_function(
    arg_1, arg_2, arg_3=80
)



In [ ]:
def fixture_function(arg_1: str, arg_2, arg_3: Optional[str] = None, arg_4:  Dict[str, int] = {}, arg_5: Tuple[int, str, float] = (1, "string", 2.0)) -> str:
    pass


expected = """fixture_function(
    arg_1, arg_2, arg_3=None, arg_4={}, arg_5=(1, 'string', 2.0)
)
"""
_signature = signature(fixture_function)
parameters = _get_parameters(_signature)
actual = _format_symbol_definition(fixture_function, parameters)
print(actual)
assert actual == expected

fixture_function(
    arg_1, arg_2, arg_3=None, arg_4={}, arg_5=(1, 'string', 2.0)
)



In [ ]:
def fixture_function(
    arg_1: int,
    arg_2: str = "default_string",
    arg_3: Dict[str, int] = {},
    arg_4: Optional[float] = None,
    arg_5: Tuple[int, str, float] = (1, "string", 2.0),
    arg_6: List[Union[int, str]] = [1, "string"],
    arg_7: Set[int] = {1,2,3},
    arg_8: str = "string"
) -> None:
    pass

expected = """fixture_function(
    arg_1,
    arg_2='default_string',
    arg_3={},
    arg_4=None,
    arg_5=(1, 'string', 2.0),
    arg_6=[1, 'string'],
    arg_7={1, 2, 3},
    arg_8='string',
)
"""

_signature = signature(fixture_function)
parameters = _get_parameters(_signature)

actual = _format_symbol_definition(fixture_function, parameters)
print(actual)
assert actual == expected

fixture_function(
    arg_1,
    arg_2='default_string',
    arg_3={},
    arg_4=None,
    arg_5=(1, 'string', 2.0),
    arg_6=[1, 'string'],
    arg_7={1, 2, 3},
    arg_8='string',
)



In [ ]:
@contextmanager
def add_tmp_path_to_sys_path(dir_):
    dir_ = Path(dir_).absolute().resolve(strict=True)
    original_path = sys.path[:]
    sys.path.insert(0, str(dir_))
    try:
        yield
    finally:
        sys.path = original_path
        
with TemporaryDirectory() as d:
    with add_tmp_path_to_sys_path(d):
        actual = sys.path[:]
        print(actual)
        assert str(Path(d).resolve()) in actual

['/tmp/tmpa2e4gvxb', '/work/fastkafka/nbs', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/home/harish/.local/lib/python3.11/site-packages', '/work/fastkafka', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages']


In [ ]:
# | export

def _get_exps(mod: str) -> Dict[str, str]:
    mf = _find_mod(mod)
    if not mf: return {}
    txt = mf.read_text()
    _def_types = ast.FunctionDef,ast.AsyncFunctionDef,ast.ClassDef
    d = {}
    for tree in ast.parse(txt).body:
        if isinstance(tree, _def_types):
            for t in L(patch_name(tree)): d[t] = f"{tree.lineno}-L{tree.end_lineno}"
        if isinstance(tree, ast.ClassDef): d.update({tree.name+"."+t2.name: f"{t2.lineno}-L{t2.end_lineno}" for t2 in tree.body if isinstance(t2, _def_types)})
    return d

In [ ]:
module_code = '''__all__ = ['FixtureClass']

from typing import *

class FixtureClass:    
    def __init__(self, attribute):
        """__init__ documentation"""
        self.attribute = attribute
    
    @classmethod
    def class_method(cls):
        """class_method documentation"""
        return cls.class_variable
    
    @staticmethod
    def static_method():
        """static_method documentation"""
        return "This is a static method"
    
    def instance_method(self):
        """instance_method documentation"""
        return "This is an instance method"
'''

with TemporaryDirectory() as d:
    my_package = f"mypackage_{random.randint(0, 1000)}"
    module_name = "mymodule"

    my_package_path = Path(d) / my_package
    my_package_path.mkdir(parents=True)

    file_path = my_package_path / f"{module_name}.py"

    with open(file_path, "w", encoding="utf-8") as file:
        file.write(module_code)

    with open((my_package_path / "__init__.py"), "w") as f:
        f.write('__version__ = "0.0.1"')

    with add_tmp_path_to_sys_path(d):
        actual = _get_exps(f"{my_package}/{module_name}.py")
        expected = {
            "FixtureClass": "5-L22",
            "FixtureClass.__init__": "6-L8",
            "FixtureClass.class_method": "11-L13",
            "FixtureClass.static_method": "16-L18",
            "FixtureClass.instance_method": "20-L22",
        }
        print(actual)
        assert actual == expected

{'FixtureClass': '5-L22', 'FixtureClass.__init__': '6-L8', 'FixtureClass.class_method': '11-L13', 'FixtureClass.static_method': '16-L18', 'FixtureClass.instance_method': '20-L22'}


In [ ]:
# | export


def _lineno(sym: str, fname: str) -> Optional[str]:
    return _get_exps(fname).get(sym, None) if fname else None


@lru_cache(None)
class CustomNbdevLookup(NbdevLookup.__wrapped__): # type: ignore
    def __init__(
        self, strip_libs: Optional[str] = None, incl_libs: Optional[str] = None, skip_mods: Optional[str] = None
    ):
        super().__init__(strip_libs, incl_libs, skip_mods)

    def code(self, sym: str) -> Optional[str]:
        "Link to source code for `sym`"
        res = self[sym]
        if not isinstance(res, tuple):
            return None
        _, py, gh = res
        line = _lineno(sym, py)
        return f"{gh}#L{line}"

In [ ]:
actual = CustomNbdevLookup().code('FastKafka')
print(actual)

pattern = r'#L\d+-L\d+'
assert re.search(pattern, actual)

https://github.com/airtai/fastkafka/blob/main/fastkafka/_application/app.py#L176-L426


In [ ]:
@contextmanager
def mock_custom_nbdev_lookup():
    with patch('__main__.CustomNbdevLookup') as MockCustomNbdevLookup:
        instance = MockCustomNbdevLookup.return_value
        instance.code.return_value = "https://github.com/airtai/fastkafka/blob/main/fastkafka/_application/app.py#L171-L425"
        yield

with mock_custom_nbdev_lookup():
    actual = CustomNbdevLookup().code("some_symbol_qualname")
    print(actual)
    expected = "https://github.com/airtai/fastkafka/blob/main/fastkafka/_application/app.py#L171-L425"
    assert actual == expected

https://github.com/airtai/fastkafka/blob/main/fastkafka/_application/app.py#L171-L425


In [ ]:
# | export


def _get_symbol_source_link(symbol: Type, lib_version: str) -> str:
    """Returns the source code link for a given symbol.

    Args:
        symbol: The symbol to get the source code link for.
        lib_version: The current version of the library.

    Returns:
        The source code link for the symbol.
    """
    symbol = symbol.fget if isinstance(symbol, property) else symbol
    source_link = CustomNbdevLookup().code(f"{symbol.__qualname__}")
    
    if source_link is None:
        return ''
    
    href = (
        source_link.replace("/blob/main/", f"/blob/{lib_version}/")
        if lib_version.replace(".", "").isdigit()
        else source_link
    )
    return f'<a href="{href}" class="link-to-source" target="_blank">View source</a>'

In [ ]:
class FixtureClass:
    def __init__(self, attribute):
        self.attribute = attribute
    
    @property
    def property_attribute(self):
        pass
    
    @classmethod
    def class_method(cls):
        pass
    
    def instance_method(self):
        pass

with mock_custom_nbdev_lookup():
    members = getmembers(FixtureClass, lambda a : isfunction(a) or ismethod(a))
    lib_version = "0.7.0"
    for m in members:
        actual = _get_symbol_source_link(m[1], lib_version)
        print(actual)
        expected = '''<a href="https://github.com/airtai/fastkafka/blob/0.7.0/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>'''
        assert actual == expected

<a href="https://github.com/airtai/fastkafka/blob/0.7.0/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>
<a href="https://github.com/airtai/fastkafka/blob/0.7.0/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>
<a href="https://github.com/airtai/fastkafka/blob/0.7.0/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>


In [ ]:
with mock_custom_nbdev_lookup():
    members = getmembers(FixtureClass, lambda a : isfunction(a) or ismethod(a))
    lib_version = "0.8.0rc0"
    for m in members:
        actual = _get_symbol_source_link(m[1], lib_version)
        print(actual)
        expected = '''<a href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>'''
        assert actual == expected

<a href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>
<a href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>
<a href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>


In [ ]:
with mock_custom_nbdev_lookup():
    members = getmembers(FixtureClass, lambda a : isfunction(a) or ismethod(a))
    lib_version = "dev"
    for m in members:
        actual = _get_symbol_source_link(m[1], lib_version)
        print(actual)
        expected = '''<a href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>'''
        assert actual == expected

<a href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>
<a href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>
<a href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>


In [ ]:
# | export


def _get_method_type(symbol: Type) -> str:
    try:
        source = getsource(symbol).strip()
    except TypeError as e:
        return ""

    first_line = source.split("\n")[0]
    return (
        f"{first_line}\n"
        if first_line
        in ["@abstractmethod", "@staticmethod", "@classmethod", "@property"]
        else ""
    )


def _get_symbol_definition(symbol: Type, header_level: int, lib_version: str) -> str:
    """Return the definition of a given symbol.

    Args:
        symbol: A function or method object to get the definition for.
        header_level: The level of the markdown header to append.
        lib_version: The current version of the library.

    Returns:
        A string representing the function definition
    """
    if isclass(symbol):
        return f"{'#'*(header_level - 1)} {symbol.__module__}.{symbol.__name__} {{#{symbol.__module__}.{symbol.__name__}}}\n\n{_get_symbol_source_link(symbol, lib_version)}\n\n"

    if isinstance(symbol, property):
        symbol = symbol.fget

    symbol_anchor = (
        f"{'#' * header_level} {symbol.__name__}"
        + f" {{#{symbol.__module__}.{'.'.join([component.strip('_') for component in symbol.__qualname__.rsplit('.', 1)])}}}\n\n"
    )

    link_to_source = f"{_get_symbol_source_link(symbol, lib_version)}\n\n"

    _signature = signature(symbol)
    parameters = _get_parameters(_signature)
    symbol_definition = f"```py\n{_get_method_type(symbol)}{_format_symbol_definition(symbol, parameters)}```\n"
    return symbol_anchor + link_to_source + symbol_definition

In [ ]:
def fixture_decorator(func):
    @functools.wraps(func)
    def wrapped_func():
        func()
    return wrapped_func

class MyClass:
    attribute = "Some Attribute"
    
    def __init__(self):
        """__init__ documentation"""
        pass

    @fixture_decorator
    def instance_method(self, a, b, c):
        """instance_method documentation"""
        pass

    @property
    def property_attribute(self, a):
        """property_attribute documentation"""
        return self.attribute

    @classmethod
    @fixture_decorator
    def class_method(cls, a):
        """class_method documentation"""
        return cls.class_variable

    @staticmethod
    def static_method(x):
        """static_method documentation"""
        pass

    @abstractmethod
    def abstract_method(self, xyz):
        """abstract_method documentation"""
        pass


with mock_custom_nbdev_lookup():
    actual = _get_symbol_definition(MyClass, 5, "dev")
    print(actual)
    assert actual == '#### __main__.MyClass {#__main__.MyClass}\n\n<a href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>\n\n'
    
    actual = _get_symbol_definition(MyClass.__init__, 5, "1.0.0")
    display(actual)
    assert (
        actual
        == '''##### __init__ {#__main__.MyClass.init}\n\n<a href="https://github.com/airtai/fastkafka/blob/1.0.0/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>\n\n```py\n__init__(\n    self\n)\n```\n'''
    )
    
    
    actual = _get_symbol_definition(MyClass.class_method, 5, "1.0.0")
    print(actual)
    assert (
        actual
        == '''##### class_method {#__main__.MyClass.class_method}\n\n<a href="https://github.com/airtai/fastkafka/blob/1.0.0/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>\n\n```py\n@classmethod\nclass_method(\n    a\n)\n```\n'''
    )

    actual = _get_symbol_definition(MyClass.abstract_method, 5, "0.2.0rc0")
    print(actual)
    assert (
        actual
        == '''##### abstract_method {#__main__.MyClass.abstract_method}\n\n<a href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>\n\n```py\n@abstractmethod\nabstract_method(\n    self, xyz\n)\n```\n'''
    )

    actual = _get_symbol_definition(MyClass.static_method, 5, "0.7.0dev")
    print(actual)
    assert (
        actual
        == '''##### static_method {#__main__.MyClass.static_method}\n\n<a href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>\n\n```py\n@staticmethod\nstatic_method(\n    x\n)\n```\n'''
    )

    actual = _get_symbol_definition(MyClass.instance_method, 5, "1.0.1-rc.0.1")
    print(actual)
    assert (
        actual
        == '''##### instance_method {#__main__.MyClass.instance_method}\n\n<a href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>\n\n```py\ninstance_method(\n    self, a, b, c\n)\n```\n'''
    )

    actual = _get_symbol_definition(MyClass.property_attribute, 5, "1.0")
    print(actual)
    assert (
        actual
        == '''##### property_attribute {#__main__.MyClass.property_attribute}\n\n<a href="https://github.com/airtai/fastkafka/blob/1.0/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>\n\n```py\n@property\nproperty_attribute(\n    self, a\n)\n```\n'''
    )

#### __main__.MyClass {#__main__.MyClass}

<a href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>




'##### __init__ {#__main__.MyClass.init}\n\n<a href="https://github.com/airtai/fastkafka/blob/1.0.0/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>\n\n```py\n__init__(\n    self\n)\n```\n'

##### class_method {#__main__.MyClass.class_method}

<a href="https://github.com/airtai/fastkafka/blob/1.0.0/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>

```py
@classmethod
class_method(
    a
)
```

##### abstract_method {#__main__.MyClass.abstract_method}

<a href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>

```py
@abstractmethod
abstract_method(
    self, xyz
)
```

##### static_method {#__main__.MyClass.static_method}

<a href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>

```py
@staticmethod
static_method(
    x
)
```

##### instance_method {#__main__.MyClass.instance_method}

<a href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>

```py
insta

In [ ]:
def fixture_function() -> None:
    pass

with mock_custom_nbdev_lookup():
    actual = _get_symbol_definition(fixture_function, 5, "0.7.0")
    expected = """    ##### fixture_function {#__main__.fixture_function}

    <a href="https://github.com/airtai/fastkafka/blob/0.7.0/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>

    ```py
    fixture_function()
    ```
    """

    print(actual)
    assert actual == textwrap.dedent(expected)

##### fixture_function {#__main__.fixture_function}

<a href="https://github.com/airtai/fastkafka/blob/0.7.0/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>

```py
fixture_function()
```



In [ ]:
TestCallable = Callable[[BaseModel], Union[Awaitable[None], None]]

def fixture_function(arg_1: str, arg_2) -> TestCallable:
    pass


with mock_custom_nbdev_lookup():
    actual = _get_symbol_definition(fixture_function, 3, "1.0")
    expected = """    ### fixture_function {#__main__.fixture_function}
    
    <a href="https://github.com/airtai/fastkafka/blob/1.0/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>

    ```py
    fixture_function(
        arg_1, arg_2
    )
    ```
    """

    print(actual)
    assert actual == textwrap.dedent(expected)

### fixture_function {#__main__.fixture_function}

<a href="https://github.com/airtai/fastkafka/blob/1.0/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>

```py
fixture_function(
    arg_1, arg_2
)
```



In [ ]:
_object = object()

def fixture_function(
    api_version="auto",
    acks=_object,
    max_poll_interval_ms=300000,
    partition_assignment_strategy=(RoundRobinPartitionAssignor,),
    isolation_level="read_uncommitted") -> None:
    pass

with mock_custom_nbdev_lookup():

    actual = _get_symbol_definition(fixture_function, 3, "dev")
    print(actual)
    assert actual is not None
    assert '''<a href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>''' in actual

### fixture_function {#__main__.fixture_function}

<a href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>

```py
fixture_function(
    api_version='auto',
    acks=<object object>,
    max_poll_interval_ms=300000,
    partition_assignment_strategy=(<class 'kafka.coordinator.assignors.roundrobin.RoundRobinPartitionAssignor'>,),
    isolation_level='read_uncommitted',
)
```



In [ ]:
def fixture_function(
    arg_1: int,
    arg_2: str = "default_string",
    arg_3: Dict[str, int] = {},
    arg_4: Optional[float] = None,
    arg_5: Tuple[int, str, float] = (1, "string", 2.0),
    arg_6: List[Union[int, str]] = [1, "string"],
    arg_7: Set[int] = {1, 2, 3},
    arg_8: Union[int, str] = "string",
) -> None:
    pass

with mock_custom_nbdev_lookup():
    actual = _get_symbol_definition(fixture_function, 1, "1.0")
    expected = """    # fixture_function {#__main__.fixture_function}
    
    <a href="https://github.com/airtai/fastkafka/blob/1.0/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>

    ```py
    fixture_function(
        arg_1,
        arg_2='default_string',
        arg_3={},
        arg_4=None,
        arg_5=(1, 'string', 2.0),
        arg_6=[1, 'string'],
        arg_7={1, 2, 3},
        arg_8='string',
    )
    ```
    """

    print(actual)
    assert actual == textwrap.dedent(expected)

# fixture_function {#__main__.fixture_function}

<a href="https://github.com/airtai/fastkafka/blob/1.0/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>

```py
fixture_function(
    arg_1,
    arg_2='default_string',
    arg_3={},
    arg_4=None,
    arg_5=(1, 'string', 2.0),
    arg_6=[1, 'string'],
    arg_7={1, 2, 3},
    arg_8='string',
)
```



In [ ]:
# | export


def _is_method(symbol: Type) -> bool:
    """Check if the given symbol is a method.

    Args:
        symbol: A function or method object to check.

    Returns:
        A boolean indicating whether the symbol is a method.
    """
    return (
        ismethod(symbol)
        or isfunction(symbol)
        or isinstance(symbol, property)
    )

In [ ]:
assert _is_method(MyClass.instance_method)
assert _is_method(MyClass.static_method)
assert _is_method(MyClass.class_method)
assert _is_method(MyClass.abstract_method)
assert _is_method(MyClass.property_attribute)
assert not _is_method(MyClass.attribute)

In [ ]:
# | export

def _get_formatted_docstring_for_symbol(
    symbol: Type,
    lib_version: str,
    header_level: int = 2
) -> str:
    """Recursively parses and get formatted docstring of a symbol.

    Args:
        symbol: A Python class or function object to parse the docstring for.
        lib_version: The current version of the library.
        header_level: The level of the markdown header to append.

    Returns:
        A formatted docstring of the symbol and its members.

    """

    def traverse(symbol: Type, contents: str, header_level: int, lib_version: str) -> str:
        """Recursively traverse the members of a symbol and append their docstrings to the provided contents string.

        Args:
            symbol: A Python class or function object to parse the docstring for.
            contents: The current formatted docstrings.
            header_level: The level of the markdown header to append.
            lib_version: The current version of the library.

        Returns:
            The updated formatted docstrings.

        """
        for x, y in getmembers(symbol):
            if not x.startswith("_") or x == "__init__":
                if _is_method(y) and y.__doc__ is not None:
                    contents += f"{_get_symbol_definition(y, header_level, lib_version)}\n{_docstring_to_markdown(y)}"
                elif isclass(y) and y.__doc__ is not None and not x.startswith("_"):
                    contents += f"{_get_symbol_definition(y, header_level+1, lib_version)}\n{_docstring_to_markdown(y)}"
                    contents = traverse(y, contents, header_level+1, lib_version)
        return contents

    contents = (
        f"{_get_symbol_definition(symbol, header_level+1, lib_version)}\n{_docstring_to_markdown(symbol)}"
        if symbol.__doc__ is not None
        else ""
    )
    if isclass(symbol):
        contents = traverse(symbol, contents, header_level+1, lib_version)
    return contents

In [ ]:
class FixtureClass:
    """Fixture documentation"""    
    class_variable = 10
    
    def __init__(self, attribute):
        """__init__ documentation"""
        self.attribute = attribute
    
    @property
    def property_attribute(self):
        """property_attribute documentation"""
        return self.attribute
    
    @classmethod
    def class_method(cls):
        """class_method documentation"""
        return cls.class_variable
    
    @staticmethod
    @fixture_decorator
    def static_method():
        """static_method documentation"""
        return "This is a static method"
    
    @fixture_decorator
    def instance_method(self):
        """instance_method documentation"""
        return "This is an instance method"
    
    def __str__(self):
        """__str__ documentation"""
        return f"MyClass instance with attribute: {self.attribute}"
    
    @abstractmethod
    def abstract_method(self):
        """abstract_method documentation"""
        pass
    
    class NestedClass:
        """NestedClass documentation"""
        def nested_method(self):
            """nested_method documentation"""
            return "This is a method in the nested class"
        
        class NestedNestedClass:
            """NestedNestedClass documentation"""
            
            def nested_nested_method(self):
                """nested_nested_method documentation"""
                return "This is a method in the nested_nested class"





with mock_custom_nbdev_lookup():
    actual = _get_formatted_docstring_for_symbol(FixtureClass, "0.1.0")
    expected = '''## __main__.FixtureClass {#__main__.FixtureClass}\n\n<a href="https://github.com/airtai/fastkafka/blob/0.1.0/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>\n\n\nFixture documentation\n\n### __main__.NestedClass {#__main__.NestedClass}\n\n<a href="https://github.com/airtai/fastkafka/blob/0.1.0/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>\n\n\nNestedClass documentation\n\n#### __main__.NestedNestedClass {#__main__.NestedNestedClass}\n\n<a href="https://github.com/airtai/fastkafka/blob/0.1.0/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>\n\n\nNestedNestedClass documentation\n\n##### nested_nested_method {#__main__.FixtureClass.NestedClass.NestedNestedClass.nested_nested_method}\n\n<a href="https://github.com/airtai/fastkafka/blob/0.1.0/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>\n\n```py\nnested_nested_method(\n    self\n)\n```\n\nnested_nested_method documentation\n\n#### nested_method {#__main__.FixtureClass.NestedClass.nested_method}\n\n<a href="https://github.com/airtai/fastkafka/blob/0.1.0/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>\n\n```py\nnested_method(\n    self\n)\n```\n\nnested_method documentation\n\n### __init__ {#__main__.FixtureClass.init}\n\n<a href="https://github.com/airtai/fastkafka/blob/0.1.0/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>\n\n```py\n__init__(\n    self, attribute\n)\n```\n\n__init__ documentation\n\n### abstract_method {#__main__.FixtureClass.abstract_method}\n\n<a href="https://github.com/airtai/fastkafka/blob/0.1.0/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>\n\n```py\n@abstractmethod\nabstract_method(\n    self\n)\n```\n\nabstract_method documentation\n\n### class_method {#__main__.FixtureClass.class_method}\n\n<a href="https://github.com/airtai/fastkafka/blob/0.1.0/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>\n\n```py\n@classmethod\nclass_method()\n```\n\nclass_method documentation\n\n### instance_method {#__main__.FixtureClass.instance_method}\n\n<a href="https://github.com/airtai/fastkafka/blob/0.1.0/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>\n\n```py\ninstance_method(\n    self\n)\n```\n\ninstance_method documentation\n\n### property_attribute {#__main__.FixtureClass.property_attribute}\n\n<a href="https://github.com/airtai/fastkafka/blob/0.1.0/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>\n\n```py\n@property\nproperty_attribute(\n    self\n)\n```\n\nproperty_attribute documentation\n\n### static_method {#__main__.FixtureClass.static_method}\n\n<a href="https://github.com/airtai/fastkafka/blob/0.1.0/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>\n\n```py\n@staticmethod\nstatic_method()\n```\n\nstatic_method documentation\n\n'''
    print(actual)
    
    assert actual == expected

## __main__.FixtureClass {#__main__.FixtureClass}

<a href="https://github.com/airtai/fastkafka/blob/0.1.0/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>


Fixture documentation

### __main__.NestedClass {#__main__.NestedClass}

<a href="https://github.com/airtai/fastkafka/blob/0.1.0/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>


NestedClass documentation

#### __main__.NestedNestedClass {#__main__.NestedNestedClass}

<a href="https://github.com/airtai/fastkafka/blob/0.1.0/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>


NestedNestedClass documentation

##### nested_nested_method {#__main__.FixtureClass.NestedClass.NestedNestedClass.nested_nested_method}

<a href="https://github.com/airtai/fastkafka/blob/0.1.0/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>

```py
nested_nested_method(
    se

In [ ]:
def fixture_function(
    arg_1: str,
    arg_2: Union[List[str], str],
    arg_3: Optional[int],
    arg_4: Optional[str] = None,
) -> str:
    """This is a one line description for the function

    Args:
        arg_1: Argument 1
        arg_2: Argument 2
        arg_3: Argument 3
        arg_4: Argument 4

    Returns:
        The concatinated string
    """
    pass


_optional_str_type = "Union[str, NoneType]" if f"{sys.version_info.major}.{sys.version_info.minor}" == "3.8" else "Optional[str]"
_optional_int_type = "Union[int, NoneType]" if f"{sys.version_info.major}.{sys.version_info.minor}" == "3.8" else "Optional[int]"
expected = "### fixture_function {#__main__.fixture_function}"+f"""

<a href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>

```py
fixture_function(
    arg_1, arg_2, arg_3, arg_4=None
)
```

This is a one line description for the function

**Parameters**:

|  Name | Type | Description | Default |
|---|---|---|---|
| `arg_1` | `str` | Argument 1 | *required* |
| `arg_2` | `Union[List[str], str]` | Argument 2 | *required* |
| `arg_3` | `{_optional_int_type}` | Argument 3 | *required* |
| `arg_4` | `{_optional_str_type}` | Argument 4 | `None` |

**Returns**:

|  Type | Description |
|---|---|
| `str` | The concatinated string |

"""


with mock_custom_nbdev_lookup():
    actual = _get_formatted_docstring_for_symbol(fixture_function, "0.1.1rc0")
    print(actual)

    assert actual == expected

### fixture_function {#__main__.fixture_function}

<a href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>

```py
fixture_function(
    arg_1, arg_2, arg_3, arg_4=None
)
```

This is a one line description for the function

**Parameters**:

|  Name | Type | Description | Default |
|---|---|---|---|
| `arg_1` | `str` | Argument 1 | *required* |
| `arg_2` | `Union[List[str], str]` | Argument 2 | *required* |
| `arg_3` | `Optional[int]` | Argument 3 | *required* |
| `arg_4` | `Optional[str]` | Argument 4 | `None` |

**Returns**:

|  Type | Description |
|---|---|
| `str` | The concatinated string |




In [ ]:
# | export


def _convert_html_style_attribute_to_jsx(contents: str) -> str:
    """Converts the inline style attributes in an HTML string to JSX compatible format.

    Args:
        contents: A string containing an HTML document or fragment.

    Returns:
        A string with inline style attributes converted to JSX compatible format.
    """
    style_regex = re.compile(r'style="(.+?)"')
    style_matches = style_regex.findall(contents)

    for style_match in style_matches:
        style_dict = {}
        styles = style_match.split(";")
        for style in styles:
            key_value = style.split(":")
            if len(key_value) == 2:
                key = re.sub(
                    r"-(.)", lambda m: m.group(1).upper(), key_value[0].strip()
                )
                value = key_value[1].strip().replace("'", '"')
                style_dict[key] = value
        replacement = "style={{"
        for key, value in style_dict.items():
            replacement += f"{key}: '{value}', "
        replacement = replacement[:-2] + "}}"
        contents = contents.replace(f'style="{style_match}"', replacement)

    return contents

In [ ]:
fixtures = [
    {
        "input": """<a
href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_components/test_dependencies.py#L28"
target="_blank" style="float:right; font-size:smaller">source</a> some text goes here <a
href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_components/test_dependencies.py#L28"
target="_blank" style="float:right; font-size:smaller">source</a>""",
        "expected": """<a
href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_components/test_dependencies.py#L28"
target="_blank" style={{float: 'right', fontSize: 'smaller'}}>source</a> some text goes here <a
href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_components/test_dependencies.py#L28"
target="_blank" style={{float: 'right', fontSize: 'smaller'}}>source</a>""",
    },
    {
        "input": '<span style="color: red;">Test</span>',
        "expected": "<span style={{color: 'red'}}>Test</span>",
    },
    {
        "input": '<div style="background-color: blue; border: 1px solid black;">Test</div>',
        "expected": "<div style={{backgroundColor: 'blue', border: '1px solid black'}}>Test</div>",
    },
    {
        "input": '<span style="font-size: 1.2rem;">Test</span>',
        "expected": "<span style={{fontSize: '1.2rem'}}>Test</span>",
    },
    {
        "input": """<pre style="white-space:pre;overflow-x:auto;line-height:normal;font-family:Menlo,'DejaVu Sans Mono',consolas,'Courier New',monospace"></pre>""",
        "expected": """<pre style={{whiteSpace: 'pre', overflowX: 'auto', lineHeight: 'normal', fontFamily: 'Menlo,"DejaVu Sans Mono",consolas,"Courier New",monospace'}}></pre>""",
    },
]

for fixture in fixtures:
    actual = _convert_html_style_attribute_to_jsx(fixture["input"])
    print("*" * 120)
    print(actual)
    assert actual == fixture["expected"], fixture["expected"]

************************************************************************************************************************
<a
href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_components/test_dependencies.py#L28"
target="_blank" style={{float: 'right', fontSize: 'smaller'}}>source</a> some text goes here <a
href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_components/test_dependencies.py#L28"
target="_blank" style={{float: 'right', fontSize: 'smaller'}}>source</a>
************************************************************************************************************************
<span style={{color: 'red'}}>Test</span>
************************************************************************************************************************
<div style={{backgroundColor: 'blue', border: '1px solid black'}}>Test</div>
************************************************************************************************************************
<span style={{fontSize: '1

In [ ]:
# | export


def _get_all_markdown_files_path(docs_path: Path) -> List[Path]:
    """Get all Markdown files in a directory and its subdirectories.

    Args:
        directory: The path to the directory to search in.

    Returns:
        A list of paths to all Markdown files found in the directory and its subdirectories.
    """
    markdown_files = [file_path for file_path in docs_path.glob("**/*.md")]
    return markdown_files

In [ ]:
with TemporaryDirectory() as d:
    module_name = "fastkafka"

    docs_path = Path(d) / "docusaurus" / "docs"
    docs_path.mkdir(parents=True)

    api_path = docs_path / "api"
    api_path.mkdir(parents=True)

    blog_path = docs_path / "blog"
    blog_path.mkdir(parents=True)

    nested_api_path = api_path / "fastKafka"
    nested_api_path.mkdir(parents=True)

    for p in [docs_path, api_path, blog_path, nested_api_path]:
        with open((p / "file.md"), "w") as f:
            f.write("sample text")

    actual = _get_all_markdown_files_path(docs_path)
    expected = [
        Path(docs_path) / "file.md",
        Path(api_path) / "file.md",
        Path(nested_api_path) / "file.md",
        Path(blog_path) / "file.md",
    ]

    print(actual)
    assert sorted(actual) == sorted(expected), expected

[Path('/tmp/tmp5fk6cf26/docusaurus/docs/file.md'), Path('/tmp/tmp5fk6cf26/docusaurus/docs/api/file.md'), Path('/tmp/tmp5fk6cf26/docusaurus/docs/api/fastKafka/file.md'), Path('/tmp/tmp5fk6cf26/docusaurus/docs/blog/file.md')]


In [ ]:
# | export


def _fix_special_symbols_in_html(contents: str) -> str:
    contents = contents.replace("”", '"')
    return contents

In [ ]:
fixture = """<a href="https://colab.research.google.com/github/airtai/fastkafka/blob/main/nbs/guides/Guide_00_FastKafka_Demo.ipynb" target=”_blank”>"""
expected = """<a href="https://colab.research.google.com/github/airtai/fastkafka/blob/main/nbs/guides/Guide_00_FastKafka_Demo.ipynb" target="_blank">"""

actual = _fix_special_symbols_in_html(fixture)
print(actual)
assert actual == expected

<a href="https://colab.research.google.com/github/airtai/fastkafka/blob/main/nbs/guides/Guide_00_FastKafka_Demo.ipynb" target="_blank">


In [ ]:
# | export

def _add_file_extension_to_link(url: str) -> str:
    """Add file extension to the last segment of a URL

    Args:
        url: A URL string.

    Returns:
        A string of the updated URL with a file extension added to the last segment of the URL.
    """
    segments = url.split("/#")[0].split("/")[-2:]
    return url.replace(f"/{segments[1]}", f"/{segments[1]}.md").replace(".md/#", ".md#")

In [ ]:
fixture = "https://airtai.github.io/fastkafka/api/fastkafka/FastKafka/#fastkafka.FastKafka"
expected = "https://airtai.github.io/fastkafka/api/fastkafka/FastKafka.md#fastkafka.FastKafka"

actual = _add_file_extension_to_link(fixture)
print(actual)
assert actual == expected

https://airtai.github.io/fastkafka/api/fastkafka/FastKafka.md#fastkafka.FastKafka


In [ ]:
fixture = "https://airtai.github.io/fastkafka/api/fastkafka/testing/ApacheKafkaBroker/#fastkafka.testing.ApacheKafkaBroker"
expected = "https://airtai.github.io/fastkafka/api/fastkafka/testing/ApacheKafkaBroker.md#fastkafka.testing.ApacheKafkaBroker"

actual = _add_file_extension_to_link(fixture)
print(actual)
assert actual == expected

https://airtai.github.io/fastkafka/api/fastkafka/testing/ApacheKafkaBroker.md#fastkafka.testing.ApacheKafkaBroker


In [ ]:
fixture = "https://github.com/airtai/sample_fastkafka_with_redpanda"
expected = "https://github.com/airtai/sample_fastkafka_with_redpanda.md"

actual = _add_file_extension_to_link(fixture)
print(actual)
assert actual == expected

https://github.com/airtai/sample_fastkafka_with_redpanda.md


In [ ]:
# | export


def _fix_symbol_links(
    contents: str, dir_prefix: str, doc_host: str, doc_baseurl: str
) -> str:
    """Fix symbol links in Markdown content.

    Args:
        contents: The Markdown content to search for symbol links.
        dir_prefix: Directory prefix to append in the relative URL.
        doc_host: The host URL for the documentation site.
        doc_baseurl: The base URL for the documentation site.

    Returns:
        str: The Markdown content with updated symbol links.
    """
    prefix = re.escape(urljoin(doc_host + "/", doc_baseurl))
    pattern = re.compile(rf"\[(.*?)\]\(({prefix}[^)]+)\)")
    matches = pattern.findall(contents)
    for match in matches:
        old_url = match[1]
        new_url = _add_file_extension_to_link(old_url).replace("/api/", "/docs/api/")
        dir_prefix = "./" if dir_prefix == "" else dir_prefix
        relative_url = dir_prefix + new_url.split("/docs/")[1]
        contents = contents.replace(old_url, relative_url)
    return contents

In [ ]:
fixture = """In the above example,
[`FastKafka`](https://airtai.github.io/fastkafka/api/fastkafka/FastKafka/#fastkafka.FastKafka)
[`FastKafka`](https://airtai.github.io/fastkafka/0.5.0/api/fastkafka/FastKafka/#fastkafka.FastKafka)
app is named as `kafka_app`
[`FastKafka`](https://airtai.github.io/fastkafka/0.5.0rc0/api/fastkafka/FastKafka/#fastkafka.FastKafka)"""

expected = """In the above example,
[`FastKafka`](../api/fastkafka/FastKafka.md#fastkafka.FastKafka)
[`FastKafka`](../api/fastkafka/FastKafka.md#fastkafka.FastKafka)
app is named as `kafka_app`
[`FastKafka`](../api/fastkafka/FastKafka.md#fastkafka.FastKafka)"""

dir_prefix = "../"
doc_host="https://airtai.github.io"
doc_baseurl="/fastkafka"
actual = _fix_symbol_links(fixture, dir_prefix, doc_host, doc_baseurl)
print(actual)
assert actual == expected

In the above example,
[`FastKafka`](../api/fastkafka/FastKafka.md#fastkafka.FastKafka)
[`FastKafka`](../api/fastkafka/FastKafka.md#fastkafka.FastKafka)
app is named as `kafka_app`
[`FastKafka`](../api/fastkafka/FastKafka.md#fastkafka.FastKafka)


In [ ]:
fixture = """In the above example,
[`FastKafka`](https://airtai.github.io/fastkafka/api/fastkafka/FastKafka/#fastkafka.FastKafka)
[`FastKafka`](https://airtai.github.io/fastkafka/0.5.0/api/fastkafka/FastKafka/#fastkafka.FastKafka)
app is named as `kafka_app`
[`FastKafka`](https://airtai.github.io/fastkafka/0.5.0rc0/api/fastkafka/FastKafka/#fastkafka.FastKafka)"""

expected = """In the above example,
[`FastKafka`](./api/fastkafka/FastKafka.md#fastkafka.FastKafka)
[`FastKafka`](./api/fastkafka/FastKafka.md#fastkafka.FastKafka)
app is named as `kafka_app`
[`FastKafka`](./api/fastkafka/FastKafka.md#fastkafka.FastKafka)"""

dir_prefix = ""
doc_host="https://airtai.github.io"
doc_baseurl="/fastkafka"
actual = _fix_symbol_links(fixture, dir_prefix, doc_host, doc_baseurl)
print(actual)
assert actual == expected

In the above example,
[`FastKafka`](./api/fastkafka/FastKafka.md#fastkafka.FastKafka)
[`FastKafka`](./api/fastkafka/FastKafka.md#fastkafka.FastKafka)
app is named as `kafka_app`
[`FastKafka`](./api/fastkafka/FastKafka.md#fastkafka.FastKafka)


In [ ]:
fixture = """The service can be tested using the
[`Tester`](https://airtai.github.io/fastkafka/api/fastkafka/testing/Tester/#fastkafka.testing.Tester)
[`Tester`](https://airtai.github.io/fastkafka/0.5.0/api/fastkafka/testing/Tester/#fastkafka.testing.Tester)
[`Tester`](https://airtai.github.io/fastkafka/dev/api/fastkafka/testing/Tester/#fastkafka.testing.Tester)
instance and we can start the Kafka
broker locally using the
[`ApacheKafkaBroker`](https://airtai.github.io/fastkafka/api/fastkafka/testing/ApacheKafkaBroker/#fastkafka.testing.ApacheKafkaBroker)."""

expected = """The service can be tested using the
[`Tester`](./api/fastkafka/testing/Tester.md#fastkafka.testing.Tester)
[`Tester`](./api/fastkafka/testing/Tester.md#fastkafka.testing.Tester)
[`Tester`](./api/fastkafka/testing/Tester.md#fastkafka.testing.Tester)
instance and we can start the Kafka
broker locally using the
[`ApacheKafkaBroker`](./api/fastkafka/testing/ApacheKafkaBroker.md#fastkafka.testing.ApacheKafkaBroker)."""

dir_prefix = ""
doc_host="https://airtai.github.io"
doc_baseurl="/fastkafka"
actual = _fix_symbol_links(fixture, dir_prefix, doc_host, doc_baseurl)
print(actual)
assert actual == expected

The service can be tested using the
[`Tester`](./api/fastkafka/testing/Tester.md#fastkafka.testing.Tester)
[`Tester`](./api/fastkafka/testing/Tester.md#fastkafka.testing.Tester)
[`Tester`](./api/fastkafka/testing/Tester.md#fastkafka.testing.Tester)
instance and we can start the Kafka
broker locally using the
[`ApacheKafkaBroker`](./api/fastkafka/testing/ApacheKafkaBroker.md#fastkafka.testing.ApacheKafkaBroker).


In [ ]:
fixture = """This is not a link to a symbol: https://www.google.com"""

expected = """This is not a link to a symbol: https://www.google.com"""

dir_prefix = ""
doc_host="https://airtai.github.io"
doc_baseurl="/fastkafka"
actual = _fix_symbol_links(fixture, dir_prefix, doc_host, doc_baseurl)
print(actual)
assert actual == expected

This is not a link to a symbol: https://www.google.com


In [ ]:
fixture = """A sample fastkafka-based library that uses Redpanda for testing, based
on this guide, can be found
[here](https://github.com/airtai/sample_fastkafka_with_redpanda)"""

expected = """A sample fastkafka-based library that uses Redpanda for testing, based
on this guide, can be found
[here](https://github.com/airtai/sample_fastkafka_with_redpanda)"""

dir_prefix = ""
doc_host="https://airtai.github.io"
doc_baseurl="/fastkafka"
actual = _fix_symbol_links(fixture, dir_prefix, doc_host, doc_baseurl)
print(actual)
assert actual == expected

A sample fastkafka-based library that uses Redpanda for testing, based
on this guide, can be found
[here](https://github.com/airtai/sample_fastkafka_with_redpanda)


In [ ]:
fixture = """To learn more about Redpanda, please visit their
[website](https://redpanda.com/) or checkout this [blog
post](https://redpanda.com/blog/redpanda-vs-kafka-performance-benchmark)
comparing Redpanda and Kafka’s performance benchmarks."""

expected = """To learn more about Redpanda, please visit their
[website](https://redpanda.com/) or checkout this [blog
post](https://redpanda.com/blog/redpanda-vs-kafka-performance-benchmark)
comparing Redpanda and Kafka’s performance benchmarks."""

dir_prefix = ""
doc_host="https://airtai.github.io"
doc_baseurl="/fastkafka"
actual = _fix_symbol_links(fixture, dir_prefix, doc_host, doc_baseurl)
print(actual)
assert actual == expected

To learn more about Redpanda, please visit their
[website](https://redpanda.com/) or checkout this [blog
post](https://redpanda.com/blog/redpanda-vs-kafka-performance-benchmark)
comparing Redpanda and Kafka’s performance benchmarks.


In [ ]:
# | export

def _get_relative_url_prefix(docs_path: Path, sub_path: Path) -> str:
    """Returns a relative url prefix from a sub path to a docs path.

    Args:
        docs_path (Path): The docs directory path.
        sub_path (Path): The sub directory path.

    Returns:
        str: A string representing the relative path from the sub path to the docs path.

    Raises:
        ValueError: If the sub path is not a descendant of the docs path.
    """
    try:
        relative_path = sub_path.relative_to(docs_path)
    except ValueError:
        raise ValueError(f"{sub_path} is not a descendant of {docs_path}")

    return (
        "../" * (len(relative_path.parts) - 1) if len(relative_path.parts) > 1 else ""
    )

In [ ]:
docs_path = Path('docusaurus/docs')

sub_path = Path('docusaurus/docs/index.md')
actual = _get_relative_url_prefix(docs_path, sub_path) 
print(actual)
assert actual == ""

sub_path = Path('docusaurus/docs/guides/Guide_31_Using_redpanda_to_test_fastkafka.md')
actual = _get_relative_url_prefix(docs_path, sub_path)
print(actual)
assert actual == "../"

sub_path = Path('docusaurus/docs/guides/tutorial/fastkafka.md')
actual = _get_relative_url_prefix(docs_path, sub_path)
print(actual)
assert actual == "../../"

with pytest.raises(ValueError) as e:
    sub_path = Path('mkdocs/docs/guides/tutorial/fastkafka.md')
    _get_relative_url_prefix(docs_path, sub_path)


../
../../


In [ ]:
# | export


def fix_invalid_syntax_in_markdown(docs_path: str) -> None:
    """Fix invalid HTML syntax in markdown files and converts inline style attributes to JSX-compatible format.

    Args:
        docs_path: The path to the root directory to search for markdown files.
    """
    cfg = get_config()
    doc_host = cfg["doc_host"]
    doc_baseurl = cfg["doc_baseurl"]

    markdown_files = _get_all_markdown_files_path(Path(docs_path))
    for file in markdown_files:
        relative_url_prefix = _get_relative_url_prefix(Path(docs_path), file)
        contents = Path(file).read_text()

        contents = _convert_html_style_attribute_to_jsx(contents)
        contents = _fix_special_symbols_in_html(contents)
        contents = _fix_symbol_links(
            contents, relative_url_prefix, doc_host, doc_baseurl
        )
        file.write_text(contents)

In [ ]:
with TemporaryDirectory() as d:
    module_name = "fastkafka"

    docs_path = Path(d) / "docusaurus" / "docs"
    docs_path.mkdir(parents=True)

    api_path = docs_path / "api"
    api_path.mkdir(parents=True)

    blog_path = docs_path / "blog"
    blog_path.mkdir(parents=True)

    nested_api_path = api_path / "fastKafka"
    nested_api_path.mkdir(parents=True)

    for p in [docs_path, api_path, blog_path, nested_api_path]:
        with open((p / "file.md"), "w") as f:
            f.write(
                """source some text goes here Test and one more tag Test
[`FastKafka`](https://airtai.github.io/fastkafka/api/fastkafka/FastKafka/#fastkafka.FastKafka)
[`Tester`](https://airtai.github.io/fastkafka/api/fastkafka/testing/Tester/#fastkafka.testing.Tester)
[here](https://github.com/airtai/sample_fastkafka_with_redpanda)
"""
            )

    fix_invalid_syntax_in_markdown(str(docs_path))
    expected = [
        """source some text goes here Test and one more tag Test
[`FastKafka`](./api/fastkafka/FastKafka.md#fastkafka.FastKafka)
[`Tester`](./api/fastkafka/testing/Tester.md#fastkafka.testing.Tester)
[here](https://github.com/airtai/sample_fastkafka_with_redpanda)
""",
        """source some text goes here Test and one more tag Test
[`FastKafka`](../api/fastkafka/FastKafka.md#fastkafka.FastKafka)
[`Tester`](../api/fastkafka/testing/Tester.md#fastkafka.testing.Tester)
[here](https://github.com/airtai/sample_fastkafka_with_redpanda)
""",
        """source some text goes here Test and one more tag Test
[`FastKafka`](../api/fastkafka/FastKafka.md#fastkafka.FastKafka)
[`Tester`](../api/fastkafka/testing/Tester.md#fastkafka.testing.Tester)
[here](https://github.com/airtai/sample_fastkafka_with_redpanda)
""",
        """source some text goes here Test and one more tag Test
[`FastKafka`](../../api/fastkafka/FastKafka.md#fastkafka.FastKafka)
[`Tester`](../../api/fastkafka/testing/Tester.md#fastkafka.testing.Tester)
[here](https://github.com/airtai/sample_fastkafka_with_redpanda)
""",
    ]

    for i, p in enumerate([docs_path, api_path, blog_path, nested_api_path]):
        with open((p / "file.md"), "r") as f:
            actual = f.read()
            print("*" * 120)
            print(actual)
            assert actual == expected[i]

************************************************************************************************************************
source some text goes here Test and one more tag Test
[`FastKafka`](./api/fastkafka/FastKafka.md#fastkafka.FastKafka)
[`Tester`](./api/fastkafka/testing/Tester.md#fastkafka.testing.Tester)
[here](https://github.com/airtai/sample_fastkafka_with_redpanda)

************************************************************************************************************************
source some text goes here Test and one more tag Test
[`FastKafka`](../api/fastkafka/FastKafka.md#fastkafka.FastKafka)
[`Tester`](../api/fastkafka/testing/Tester.md#fastkafka.testing.Tester)
[here](https://github.com/airtai/sample_fastkafka_with_redpanda)

************************************************************************************************************************
source some text goes here Test and one more tag Test
[`FastKafka`](../api/fastkafka/FastKafka.md#fastkafka.FastKafka)
[`Test

In [ ]:
# | export


def generate_markdown_docs(module_name: str, docs_path: str) -> None:
    """Generates Markdown documentation files for the symbols in the given module and save them to the given directory.

    Args:
        module_name: The name of the module to generate documentation for.
        docs_path: The path to the directory where the documentation files will be saved.
    """
    members_with_submodules = _get_submodules(module_name)
    symbols = _load_submodules(module_name, members_with_submodules)
    lib_version = get_config()["version"]
    
    for symbol in symbols:
        content = _get_formatted_docstring_for_symbol(symbol, lib_version)
        target_file_path = (
            "/".join(f"{symbol.__module__}.{symbol.__name__}".split(".")) + ".md"
        )
        with open((Path(docs_path) / "api" / target_file_path), "w") as f:
            f.write(content)

In [ ]:
@contextmanager
def mock_get_config(lib_version):
    with patch('__main__.get_config') as mock_get_config:
        mock_get_config.return_value = {"version": lib_version}
        yield
        
with mock_get_config(lib_version="0.1.0"):
    actual = get_config()["version"]
    print(actual)
    expected = "0.1.0"
    assert actual == expected

0.1.0


In [ ]:
module_code = '''

__all__ = ['FixtureClass']

from typing import *
from abc import abstractmethod

from fastcore.basics import patch

class FixtureClass:
    """Fixture documentation"""
    
    def __init__(self, attribute):
        """__init__ documentation url=HttpUrl("https://www.google.co.uk", )"""
        self.attribute = attribute
        
    def __str__(self):
        """__str__ documentation"""
        return f"MyClass instance with attribute: {self.attribute}"
    
    @property
    def property_attribute(self):
        """property_attribute documentation"""
        return self.attribute
    
    @classmethod
    def class_method(cls):
        """class_method documentation"""
        return cls.class_variable
    
    @staticmethod
    def static_method():
        """static_method documentation"""
        return "This is a static method"
    
    def instance_method(self):
        """instance_method documentation"""
        return "This is an instance method"
    
    @abstractmethod
    def abstract_method(self):
        """abstract_method documentation"""
        pass
        
@patch
def patched_method_in_same_file(self:FixtureClass, s: str) -> None: 
    """I am a patched method in the same file"""
    pass
'''

for lib_version in ["0.1.0", "0.1.1rc0"]:
    with TemporaryDirectory() as d:
        my_package = f"mypackage_{random.randint(0, 1000)}"
        module_name = "mymodule"

        docs_path = Path(d) / "docusaurus" / "docs"
        docs_path.mkdir(parents=True)

        api_path = docs_path / "api" / my_package / module_name
        api_path.mkdir(parents=True)

        my_package_path = Path(d) / my_package
        my_package_path.mkdir(parents=True)

        file_path = my_package_path / f"{module_name}.py"

        with open(file_path, "w", encoding="utf-8") as file:
            file.write(module_code)

        with open((my_package_path / "__init__.py"), "w") as f:
            f.write(f'__version__ = "{lib_version}"')

        with add_tmp_path_to_sys_path(d):
            with mock_get_config(lib_version=lib_version):
                with mock_custom_nbdev_lookup():
                    members_with_submodules = _get_submodules(my_package)
                    symbols = _load_submodules(my_package, members_with_submodules)
                    generate_markdown_docs(my_package, str(docs_path))

        with open(api_path / "FixtureClass.md", "r", encoding="utf-8") as file:
            actual = file.read()
    
    gh_tag = lib_version if lib_version.replace(".", "").isdigit() else "main"
    expected = f"## {my_package}" + ".mymodule.FixtureClass {#" + my_package + '.mymodule.FixtureClass}\n\n<a href="https://github.com/airtai/fastkafka/blob/' + gh_tag + '/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>\n\n\nFixture documentation\n\n### __init__ {#' + my_package + '.mymodule.FixtureClass.init}\n\n<a href="https://github.com/airtai/fastkafka/blob/' + gh_tag + '/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>\n\n```py\n__init__(\n    self, attribute\n)\n```\n\n__init__ documentation url=HttpUrl(" https://www.google.co.uk ", )\n\n### abstract_method {#' + my_package + '.mymodule.FixtureClass.abstract_method}\n\n<a href="https://github.com/airtai/fastkafka/blob/' + gh_tag + '/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>\n\n```py\n@abstractmethod\nabstract_method(\n    self\n)\n```\n\nabstract_method documentation\n\n### class_method {#' + my_package + '.mymodule.FixtureClass.class_method}\n\n<a href="https://github.com/airtai/fastkafka/blob/' + gh_tag + '/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>\n\n```py\n@classmethod\nclass_method()\n```\n\nclass_method documentation\n\n### instance_method {#' + my_package + '.mymodule.FixtureClass.instance_method}\n\n<a href="https://github.com/airtai/fastkafka/blob/' + gh_tag + '/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>\n\n```py\ninstance_method(\n    self\n)\n```\n\ninstance_method documentation\n\n### patched_method_in_same_file {#' + my_package + '.mymodule.FixtureClass.patched_method_in_same_file}\n\n<a href="https://github.com/airtai/fastkafka/blob/' + gh_tag + '/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>\n\n```py\npatched_method_in_same_file(\n    self, s\n)\n```\n\nI am a patched method in the same file\n\n### property_attribute {#' + my_package + '.mymodule.FixtureClass.property_attribute}\n\n<a href="https://github.com/airtai/fastkafka/blob/' + gh_tag + '/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>\n\n```py\n@property\nproperty_attribute(\n    self\n)\n```\n\nproperty_attribute documentation\n\n### static_method {#' + my_package + '.mymodule.FixtureClass.static_method}\n\n<a href="https://github.com/airtai/fastkafka/blob/' + gh_tag + '/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>\n\n```py\n@staticmethod\nstatic_method()\n```\n\nstatic_method documentation\n\n'
    print(actual)
    assert actual == expected

## mypackage_22.mymodule.FixtureClass {#mypackage_22.mymodule.FixtureClass}

<a href="https://github.com/airtai/fastkafka/blob/0.1.0/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>


Fixture documentation

### __init__ {#mypackage_22.mymodule.FixtureClass.init}

<a href="https://github.com/airtai/fastkafka/blob/0.1.0/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>

```py
__init__(
    self, attribute
)
```

__init__ documentation url=HttpUrl(" https://www.google.co.uk ", )

### abstract_method {#mypackage_22.mymodule.FixtureClass.abstract_method}

<a href="https://github.com/airtai/fastkafka/blob/0.1.0/fastkafka/_application/app.py#L171-L425" class="link-to-source" target="_blank">View source</a>

```py
@abstractmethod
abstract_method(
    self
)
```

abstract_method documentation

### class_method {#mypackage_22.mymodule.FixtureClass.class_method}

<a href="https://github.com/airtai/fastkaf

In [ ]:
# | export

def _parse_lines(lines: List[str]) -> Tuple[List[str], int]:
    """Parse a list of lines and return a tuple containing a list of filenames and an index indicating how many lines to skip.

    Args:
        lines: A list of strings representing lines of input text.

    Returns:
        A tuple containing a list of strings representing the filenames extracted
        from links in the lines and an integer representing the number of lines to skip.
    """
    index = next(
        (i for i, line in enumerate(lines) if not line.strip().startswith("- [")),
        len(lines),
    )
    return [line.split("(")[1][:-4] for line in lines[:index]], index

In [ ]:
fixture = ['            - [json_encoder](api/fastkafka/encoder/json_encoder.md)', '        - testing', '            - [ApacheKafkaBroker](api/fastkafka/testing/ApacheKafkaBroker.md)', '            - [LocalRedpandaBroker](api/fastkafka/testing/LocalRedpandaBroker.md)', '            - [Tester](api/fastkafka/testing/Tester.md)']
expected = (['api/fastkafka/encoder/json_encoder'], 1)

actual = _parse_lines(fixture)
print(actual)

assert actual == expected, actual

(['api/fastkafka/encoder/json_encoder'], 1)


In [ ]:

fixture = ['            - [ApacheKafkaBroker](api/fastkafka/testing/ApacheKafkaBroker.md)', '            - [LocalRedpandaBroker](api/fastkafka/testing/LocalRedpandaBroker.md)', '            - [Tester](api/fastkafka/testing/Tester.md)']
expected = (['api/fastkafka/testing/ApacheKafkaBroker', 'api/fastkafka/testing/LocalRedpandaBroker', 'api/fastkafka/testing/Tester'], 3)

actual = _parse_lines(fixture)
print(actual)

assert actual == expected, actual

(['api/fastkafka/testing/ApacheKafkaBroker', 'api/fastkafka/testing/LocalRedpandaBroker', 'api/fastkafka/testing/Tester'], 3)


In [ ]:
# | export

def _parse_section(text: str, ignore_first_line: bool = False) -> List[Any]:
    """Parse the given section contents and return a list of file names in the expected format.

    Args:
        text: A string representing the contents of a file.
        ignore_first_line: Flag indicating whether to ignore the first line extracting the section contents.

    Returns:
        A list of filenames in the expected format
    """
    pattern = r"\[.*?\]\((.*?)\)|\[(.*?)\]\[(.*?)\]"
    lines = text.split("\n")[1:] if ignore_first_line else text.split("\n")
    ret_val = []
    index = 0
    while index < len(lines):
        line = lines[index]
        match = re.search(pattern, line.strip())
        if match is not None:
            ret_val.append(match.group(1).split(".md")[0])
            index += 1
        elif line.strip() != "":
            value, skip_lines = _parse_lines(lines[index + 1 :])
            ret_val.append({line.replace("-", "").strip(): value})
            index += skip_lines + 1
        else:
            index += 1
    return ret_val

In [ ]:
fixture = """    - fastkafka
        - [FastKafka](api/fastkafka/FastKafka.md)
        - [KafkaEvent](api/fastkafka/KafkaEvent.md)
        - encoder
            - [json_encoder](api/fastkafka/encoder/json_encoder.md)
        - testing
            - [ApacheKafkaBroker](api/fastkafka/testing/ApacheKafkaBroker.md)
            - [LocalRedpandaBroker](api/fastkafka/testing/LocalRedpandaBroker.md)
            - [Tester](api/fastkafka/testing/Tester.md)
"""

expected = [
    "api/fastkafka/FastKafka",
    "api/fastkafka/KafkaEvent",
    {"encoder": ["api/fastkafka/encoder/json_encoder"]},
    {
        "testing": [
            "api/fastkafka/testing/ApacheKafkaBroker",
            "api/fastkafka/testing/LocalRedpandaBroker",
            "api/fastkafka/testing/Tester",
        ]
    }
]

ignore_first_line = True
actual = _parse_section(fixture, ignore_first_line)
display(actual)
assert actual == expected

['api/fastkafka/FastKafka',
 'api/fastkafka/KafkaEvent',
 {'encoder': ['api/fastkafka/encoder/json_encoder']},
 {'testing': ['api/fastkafka/testing/ApacheKafkaBroker',
   'api/fastkafka/testing/LocalRedpandaBroker',
   'api/fastkafka/testing/Tester']}]

In [ ]:
fixture = """    - Writing services
        - [@consumes basics](guides/Guide_11_Consumes_Basics.md)
        - [@consumes basics](guides/Guide_21_Produces_Basics.md)
    - Testing
        - [Using Redpanda to test FastKafka](guides/Guide_31_Using_redpanda_to_test_fastkafka.md)
"""

expected = [
    {
        "Writing services": [
            "guides/Guide_11_Consumes_Basics",
            "guides/Guide_21_Produces_Basics",
        ],
    },
    {
        "Testing": ["guides/Guide_31_Using_redpanda_to_test_fastkafka"],
    },
]

actual = _parse_section(fixture)
display(actual)
assert actual == expected

[{'Writing services': ['guides/Guide_11_Consumes_Basics',
   'guides/Guide_21_Produces_Basics']},
 {'Testing': ['guides/Guide_31_Using_redpanda_to_test_fastkafka']}]

In [ ]:
# | export

def _get_section_from_markdown(
    markdown_text: str, section_header: str
) -> Optional[str]:
    """Get the contents of the section header from the given markdown text

    Args:
        markdown_text: A string containing the markdown text to extract the section from.
        section_header: A string representing the header of the section to extract.

    Returns:
        A string representing the contents of the section header if the section header
        is present in the markdown text, else None
    """
    pattern = re.compile(rf"^- {section_header}\n((?:\s+- .*\n)+)", re.M)
    match = pattern.search(markdown_text)
    return match.group(1) if match else None

In [ ]:
summary = """- [FastKafka](index.md)
- Guides
    - Writing services
        - [@consumes basics](guides/Guide_11_Consumes_Basics.md)
        - [@consumes basics](guides/Guide_11_Consumes_Basics.md)
    - Testing
        - [Using Redpanda to test FastKafka](guides/Guide_31_Using_redpanda_to_test_fastkafka.md)
- API
    - fastkafka
        - [FastKafka](api/fastkafka/FastKafka.md)
        - [KafkaEvent](api/fastkafka/KafkaEvent.md)
        - encoder
            - [json_encoder](api/fastkafka/encoder/json_encoder.md)
        - testing
            - [ApacheKafkaBroker](api/fastkafka/testing/ApacheKafkaBroker.md)
            - [LocalRedpandaBroker](api/fastkafka/testing/LocalRedpandaBroker.md)
            - [Tester](api/fastkafka/testing/Tester.md)
- CLI
    - [fastkafka](cli/fastkafka.md)
    - [run_fastkafka_server_process](cli/run_fastkafka_server_process.md)
- [Releases](CHANGELOG.md)"""

section_header = "API"
expected = """    - fastkafka
        - [FastKafka](api/fastkafka/FastKafka.md)
        - [KafkaEvent](api/fastkafka/KafkaEvent.md)
        - encoder
            - [json_encoder](api/fastkafka/encoder/json_encoder.md)
        - testing
            - [ApacheKafkaBroker](api/fastkafka/testing/ApacheKafkaBroker.md)
            - [LocalRedpandaBroker](api/fastkafka/testing/LocalRedpandaBroker.md)
            - [Tester](api/fastkafka/testing/Tester.md)
"""
actual = _get_section_from_markdown(summary, section_header)
print(actual)
assert actual == expected

    - fastkafka
        - [FastKafka](api/fastkafka/FastKafka.md)
        - [KafkaEvent](api/fastkafka/KafkaEvent.md)
        - encoder
            - [json_encoder](api/fastkafka/encoder/json_encoder.md)
        - testing
            - [ApacheKafkaBroker](api/fastkafka/testing/ApacheKafkaBroker.md)
            - [LocalRedpandaBroker](api/fastkafka/testing/LocalRedpandaBroker.md)
            - [Tester](api/fastkafka/testing/Tester.md)



In [ ]:
section_header = "CLI"
expected = """    - [fastkafka](cli/fastkafka.md)
    - [run_fastkafka_server_process](cli/run_fastkafka_server_process.md)
"""
actual = _get_section_from_markdown(summary, section_header)
print(actual)
assert actual == expected

    - [fastkafka](cli/fastkafka.md)
    - [run_fastkafka_server_process](cli/run_fastkafka_server_process.md)



In [ ]:
section_header = "Guides"
expected = """    - Writing services
        - [@consumes basics](guides/Guide_11_Consumes_Basics.md)
        - [@consumes basics](guides/Guide_11_Consumes_Basics.md)
    - Testing
        - [Using Redpanda to test FastKafka](guides/Guide_31_Using_redpanda_to_test_fastkafka.md)
"""
actual = _get_section_from_markdown(summary, section_header)
print(actual)
assert actual == expected

    - Writing services
        - [@consumes basics](guides/Guide_11_Consumes_Basics.md)
        - [@consumes basics](guides/Guide_11_Consumes_Basics.md)
    - Testing
        - [Using Redpanda to test FastKafka](guides/Guide_31_Using_redpanda_to_test_fastkafka.md)



In [ ]:
section_header = "Invalid Section"
expected = None
actual = _get_section_from_markdown(summary, section_header)
print(actual)
assert actual == expected

None


In [ ]:
# | export


def generate_sidebar(
    summary_file: str = "./docusaurus/docs/SUMMARY.md",
    summary: str = "",
    target: str = "./docusaurus/sidebars.js",
) -> None:
    """
    Generate a sidebar js file for a Docusaurus documentation site based on a SUMMARY.md file.

    Args:
        summary_file: The path to the SUMMARY.md file containing the documentation structure.
            Default is "./docusaurus/docs/SUMMARY.md".
        summary: An optional summary string.
            Default is an empty string.
        target: The path to the target sidebar js file to be generated.
            Default is "./docusaurus/sidebars.js".

    Returns:
        None: The function does not return any value directly, but it generates a sidebar file.

    Raises:
        FileNotFoundError: If the specified `summary_file` does not exist.
    """
    with open(summary_file, "r") as stream, open(target, "w") as target_stream:
        summary_contents = stream.read()

        guides_summary = _get_section_from_markdown(summary_contents, "Guides")
        parsed_guides = _parse_section(guides_summary)  # type: ignore

        api_summary = _get_section_from_markdown(summary_contents, "API")
        parsed_api = _parse_section(api_summary, True)  # type: ignore

        cli_summary = _get_section_from_markdown(summary_contents, "CLI")
        parsed_cli = _parse_section(cli_summary)  # type: ignore

        target_stream.write(
            """module.exports = {
tutorialSidebar: [
    'index', {'Guides': 
    """
            + str(parsed_guides)
            + "},"
            + "{'API': ["
            + str(parsed_api)[1:-1]
            + "]},"
            + "{'CLI': "
            + str(parsed_cli)
            + "},"
            + """
    "LICENSE",
    "CONTRIBUTING",
    "CHANGELOG",
],
};"""
        )

In [ ]:
summary = """- [FastKafka](index.md)
- Guides
    - Writing services
        - [Lifespan Events](guides/Guide_05_Lifespan_Handler.md)
        - [Encoding and Decoding Kafka Messages with FastKafka](guides/Guide_07_Encoding_and_Decoding_Messages_with_FastKafka.md)
    - Testing
        - [Using Redpanda to test FastKafka](guides/Guide_31_Using_redpanda_to_test_fastkafka.md)
    - Documentation generation
        - [Deploy FastKafka docs to GitHub Pages](guides/Guide_04_Github_Actions_Workflow.md)
- API
    - fastkafka
        - [FastKafka](api/fastkafka/FastKafka.md)
        - [KafkaEvent](api/fastkafka/KafkaEvent.md)
        - encoder
            - [AvroBase](api/fastkafka/encoder/AvroBase.md)
            - [json_decoder](api/fastkafka/encoder/json_decoder.md)
            - [json_encoder](api/fastkafka/encoder/json_encoder.md)
        - testing
            - [ApacheKafkaBroker](api/fastkafka/testing/ApacheKafkaBroker.md)
- CLI
    - [fastkafka](cli/fastkafka.md)
    - [run_fastkafka_server_process](cli/run_fastkafka_server_process.md)
- [Releases](CHANGELOG.md)"""

with TemporaryDirectory() as directory:
    with open(directory + "/SUMMARY.md", "w") as stream:
        stream.write(summary)

    generate_sidebar(
        summary_file=directory + "/SUMMARY.md", target=directory + "/test.js"
    )

    with open(directory + "/test.js") as stream:
        stream = stream.read()

print(stream)
assert (
    stream
    == """module.exports = {
tutorialSidebar: [
    'index', {'Guides': 
    [{'Writing services': ['guides/Guide_05_Lifespan_Handler', 'guides/Guide_07_Encoding_and_Decoding_Messages_with_FastKafka']}, {'Testing': ['guides/Guide_31_Using_redpanda_to_test_fastkafka']}, {'Documentation generation': ['guides/Guide_04_Github_Actions_Workflow']}]},{'API': ['api/fastkafka/FastKafka', 'api/fastkafka/KafkaEvent', {'encoder': ['api/fastkafka/encoder/AvroBase', 'api/fastkafka/encoder/json_decoder', 'api/fastkafka/encoder/json_encoder']}, {'testing': ['api/fastkafka/testing/ApacheKafkaBroker']}]},{'CLI': ['cli/fastkafka', 'cli/run_fastkafka_server_process']},
    "LICENSE",
    "CONTRIBUTING",
    "CHANGELOG",
],
};"""
), stream

module.exports = {
tutorialSidebar: [
    'index', {'Guides': 
    [{'Writing services': ['guides/Guide_05_Lifespan_Handler', 'guides/Guide_07_Encoding_and_Decoding_Messages_with_FastKafka']}, {'Testing': ['guides/Guide_31_Using_redpanda_to_test_fastkafka']}, {'Documentation generation': ['guides/Guide_04_Github_Actions_Workflow']}]},{'API': ['api/fastkafka/FastKafka', 'api/fastkafka/KafkaEvent', {'encoder': ['api/fastkafka/encoder/AvroBase', 'api/fastkafka/encoder/json_decoder', 'api/fastkafka/encoder/json_encoder']}, {'testing': ['api/fastkafka/testing/ApacheKafkaBroker']}]},{'CLI': ['cli/fastkafka', 'cli/run_fastkafka_server_process']},
    "LICENSE",
    "CONTRIBUTING",
    "CHANGELOG",
],
};


In [ ]:
# | export

def _get_markdown_filenames_from_sidebar(sidebar_file_path: str) -> List[str]:
    """Get a list of Markdown filenames included in the sidebar.

    Args:
        sidebar_file_path: The path to the sidebar file.

    Returns:
        A list of Markdown filenames included in the sidebar.
    """
    with open(sidebar_file_path, "r") as file:
        file_content = file.read()

        pattern = r"tutorialSidebar:\s*(\[.*\])\s*,\s*\n?\s*};"
        match = re.search(pattern, file_content, re.DOTALL)
        all_sidebar_files = ast.literal_eval(match.group(1)) if match else []
        markdown_filenames = [
            f"{v}.md" for v in all_sidebar_files if isinstance(v, str)
        ]
        return markdown_filenames

In [ ]:
with TemporaryDirectory() as d:
    docs_path = Path(d) / "docusaurus" / "docs"
    docs_path.mkdir(parents=True)

    sidebar_file_path = Path(d) / "docusaurus" / "sidebar.js"
    with open(sidebar_file_path, "w") as f:
        f.write(
            """module.exports = {
tutorialSidebar: [
    'index', {'Guides': 
    [{'Writing services': ['guides/Guide_05_Lifespan_Handler', 'guides/Guide_07_Encoding_and_Decoding_Messages_with_FastKafka']}, {'Testing': ['guides/Guide_31_Using_redpanda_to_test_fastkafka']}, {'Documentation generation': ['guides/Guide_04_Github_Actions_Workflow']}]},{'API': ['api/fastkafka/FastKafka', 'api/fastkafka/KafkaEvent', {'encoder': ['api/fastkafka/encoder/AvroBase', 'api/fastkafka/encoder/json_decoder', 'api/fastkafka/encoder/json_encoder']}, {'testing': ['api/fastkafka/testing/ApacheKafkaBroker']}]},{'CLI': ['cli/fastkafka', 'cli/run_fastkafka_server_process']},
    "LICENSE",
    "CONTRIBUTING",
],
};"""
        )
        
    expected = ["index.md", "LICENSE.md", "CONTRIBUTING.md"]
    actual = _get_markdown_filenames_from_sidebar(str(sidebar_file_path))
    print(sorted(actual))
    assert sorted(actual) == sorted(expected), actual

['CONTRIBUTING.md', 'LICENSE.md', 'index.md']


In [ ]:
# | export


def _delete_files(files: List[Path]) -> None:
    """Deletes a list of files.

    Args:
        files: A list of Path objects representing the files to be deleted.

    Raises:
        OSError: If an error occurs while deleting a file.

    """
    for file in files:
        try:
            file.unlink()
        except OSError as e:
            typer.echo(
                f"Error deleting files from docusaurus/docs directory. Could not delete file: {file} - {e}"
            )

In [ ]:
with TemporaryDirectory() as d:
    sample_file = Path(d) / "docusaurus" / "docs" / "t.txt"
    _delete_files([sample_file])

Error deleting files from docusaurus/docs directory. Could not delete file: /tmp/tmpwreq2fea/docusaurus/docs/t.txt - [Errno 2] No such file or directory: '/tmp/tmpwreq2fea/docusaurus/docs/t.txt'


In [ ]:
with TemporaryDirectory() as d:
    docs_path = Path(d) / "docusaurus" / "docs"
    docs_path.mkdir(parents=True)
    
    sample_file = docs_path / "file.md"
    with open(sample_file, "w") as f:
        f.write("sample text")

    _delete_files([sample_file])
    actual = [file_path for file_path in Path(docs_path).glob("*.md")]
    print(actual)
    assert actual == []

[]


In [ ]:
# | export

def delete_unused_markdown_files_from_sidebar(
    docs_path: str, sidebar_file_path: str
) -> None:
    """Delete the markdown files from the docs directory that are not present in the sidebar.

    Args:
        docs_path: Path to the directory containing the markdown files.
        sidebar_file_path: Path to the sidebar file.
    """
    md_filenames_in_sidebar = _get_markdown_filenames_from_sidebar(
        str(sidebar_file_path)
    )
    if len(md_filenames_in_sidebar) > 0:
        all_md_files_in_docs_dir = [
            file_path for file_path in Path(docs_path).glob("*.md")
        ]
        md_files_in_sidebar = [Path(docs_path) / f for f in md_filenames_in_sidebar]
        md_files_to_delete = list(
            set(all_md_files_in_docs_dir) - set(md_files_in_sidebar)
        )
        _delete_files(md_files_to_delete)

In [ ]:
with TemporaryDirectory() as d:
    docs_path = Path(d) / "docusaurus" / "docs"
    docs_path.mkdir(parents=True)

    api_path = docs_path / "api"
    api_path.mkdir(parents=True)

    nested_api_path = api_path / "fastKafka"
    nested_api_path.mkdir(parents=True)

    for p in [api_path, nested_api_path]:
        with open((p / "file.md"), "w") as f:
            f.write("sample text")

    markdown_files = [
        docs_path / "index.md",
        docs_path / "LICENSE.md",
        docs_path / "CONTRIBUTING.md",
        docs_path / "Non_sidebar_file_1.md",
        docs_path / "Non_sidebar_file_2.md",
        docs_path / "Non_sidebar_file_3.md",
    ]

    for f in markdown_files:
        with open(f, "w") as f:
            f.write("sample text")

    sidebar_file_path = Path(d) / "docusaurus" / "sidebar.js"
    with open(sidebar_file_path, "w") as f:
        f.write(
            """module.exports = {
tutorialSidebar: [
    'index', {'Guides': 
    [{'Writing services': ['guides/Guide_05_Lifespan_Handler', 'guides/Guide_07_Encoding_and_Decoding_Messages_with_FastKafka']}, {'Testing': ['guides/Guide_31_Using_redpanda_to_test_fastkafka']}, {'Documentation generation': ['guides/Guide_04_Github_Actions_Workflow']}]},{'API': ['api/fastkafka/FastKafka', 'api/fastkafka/KafkaEvent', {'encoder': ['api/fastkafka/encoder/AvroBase', 'api/fastkafka/encoder/json_decoder', 'api/fastkafka/encoder/json_encoder']}, {'testing': ['api/fastkafka/testing/ApacheKafkaBroker']}]},{'CLI': ['cli/fastkafka', 'cli/run_fastkafka_server_process']},
    "LICENSE",
    "CONTRIBUTING",
],
};"""
        )
        
    expected = [
        docs_path / "index.md",
        docs_path / "LICENSE.md",
        docs_path / "CONTRIBUTING.md",
    ]

    delete_unused_markdown_files_from_sidebar(str(docs_path), str(sidebar_file_path))
    actual = [file_path for file_path in Path(docs_path).glob("*.md")]
    print(sorted(actual))
    assert sorted(actual) == sorted(expected), actual

[Path('/tmp/tmpuqs4rolt/docusaurus/docs/CONTRIBUTING.md'), Path('/tmp/tmpuqs4rolt/docusaurus/docs/LICENSE.md'), Path('/tmp/tmpuqs4rolt/docusaurus/docs/index.md')]
